# MHKiT Power Module  
The following example uses the [MHKiT power module](https://mhkit-software.github.io/MHKiT/mhkit-python/api.power.html) to compute power, instantaneous frequency, and harmonics from time-series of voltage and current. 

Start by importing the necessary python packages and MHKiT module.

In [7]:
import numpy as np
import pandas as pd
from mhkit import power, utils
from mhkit.dolfyn.tools.fft import fft_frequency, cpsd_quasisync_1D, psd_1D
import matplotlib.pyplot as plt

## Load Power Data from HIOKI PQ3198 Power Quality Analyzer
We will begin by reading in time-series data of measured three phase (a, b, and c) voltage and current from the HIOKI PQ3198 Power Quality Analyzer.

**Wiring Configuration: 3P3W3M (3-Phase, 3-Wire, 3-Meter)**
- This configuration measures line-to-line voltages and line currents
- No neutral measurements are available in this setup

Note: IEC TS 62600-30 requires power quality assessments on a minimum of 10 minute time-series data, but for this example a shorter time-series is used. 


In [8]:
# Analyze the header structure and create channel mapping for 3P3W3M configuration
print("=== HIOKI PQ3198 Data Structure Analysis ===")
print(f"Total data points: {df_clean.size}")

# Reference the Hioki PQ3198 module header information for guidance
# Typical PQ3198 harmonic analysis includes:
# - Fundamental frequency components
# - Harmonic components (up to 50th harmonic typically)
# - Phase angles
# - THD values

# Analyze the data distribution
# Convert DataFrame to numpy array and flatten for analysis
all_values = df_clean.values.flatten()
valid_data = [val for val in all_values if not np.isnan(val)]
print(f"Valid data points: {len(valid_data)}")

try:
    if len(valid_data) > 0:
        print(f"Data range: {min(valid_data):.3f} to {max(valid_data):.3f}")
    else:
        print("Data range: No valid data available (empty sequence)")
except ValueError as e:
    print(f"Error calculating data range: {e}")

# Look for patterns in the data
# Values around 100.00E+00 might indicate percentage values (100%)
# Values around 0.12E+00 might indicate harmonic magnitudes
# Values with large ranges might indicate phase angles

# Identify potential channel groups based on data patterns
print("\n=== Data Pattern Analysis ===")

# Group data by magnitude ranges with try-except to handle empty sequences
try:
    if len(valid_data) > 0:
        small_values = [val for val in valid_data if abs(val) < 1.0]
        medium_values = [val for val in valid_data if 1.0 <= abs(val) < 50.0]
        large_values = [val for val in valid_data if abs(val) >= 50.0]

        print(f"Small values (< 1.0): {len(small_values)} points")
        print(f"Medium values (1.0-50.0): {len(medium_values)} points") 
        print(f"Large values (>= 50.0): {len(large_values)} points")

        # Look for 100% values (likely fundamental components)
        fundamental_values = [val for val in valid_data if abs(val - 100.0) < 0.1]
        print(f"Values near 100% (fundamental): {len(fundamental_values)} points")

        # Look for phase angle values (typically in degrees, large range)
        phase_angle_values = [val for val in valid_data if abs(val) > 100 and abs(val) < 200]
        print(f"Potential phase angle values: {len(phase_angle_values)} points")
    else:
        print("No valid data points available for pattern analysis")
        small_values = []
        medium_values = []
        large_values = []
        fundamental_values = []
        phase_angle_values = []
except Exception as e:
    print(f"Error in pattern analysis: {e}")
    small_values = []
    medium_values = []
    large_values = []
    fundamental_values = []
    phase_angle_values = []


=== HIOKI PQ3198 Data Structure Analysis ===
Total data points: 6013440
Valid data points: 6009505
Data range: -499.440 to 90800000.000

=== Data Pattern Analysis ===
Small values (< 1.0): 4163303 points
Medium values (1.0-50.0): 605395 points
Large values (>= 50.0): 1240807 points
Values near 100% (fundamental): 33376 points
Potential phase angle values: 758205 points


In [9]:
# Read in time-series data from HIOKI PQ3198 Power Quality Analyzer
# The CSV file contains harmonic analysis data in scientific notation format
csv_path = "HIOKI PQ Power Spectrum Sample Datasets/Sample_PQ3198/PQ3198_Dip/CSV/19072900_Dip.csv"

print("Loading HIOKI PQ3198 Power Quality Data...")
print(f"File path: {csv_path}")

# According to inspection with LibreOffice Calc, the first row of the CSV is the header.
# So read with header=0 to use the actual column headers.
try:
    df_raw = pd.read_csv(csv_path, header=0)
except ValueError as e:
    print("ValueError encountered while reading the CSV file:")

    # Custom message for CSVs that are truly without columns
    print(
        "ValueError: Cannot describe a DataFrame without columns.\n"
        "It appears the CSV may be empty or malformed (no columns detected).\n"
        "Please check that the file contains valid header and data rows.\n"
        "If using LibreOffice, ensure that the file is saved in CSV format with the first row as header."
    )
    raise

print(f"Raw data shape: {df_raw.shape}")
print(f"Number of columns: {df_raw.shape[1]}")

# Display first few values to understand the data structure
print("\nFirst 20 column names from the CSV:")
print(df_raw.columns[:20].tolist())
print("\nFirst row values from the CSV:")
print(df_raw.iloc[0, :20].values)

# The data appears to be in scientific notation format (e.g., +0.12E+00)
# Let's process the data for analysis.
def parse_scientific_notation(value):
    """Convert scientific notation string to float"""
    try:
        return float(str(value).strip())
    except (ValueError, TypeError):
        return np.nan

# Parse all values in DataFrame (applies everywhere)
df_numeric = df_raw.applymap(parse_scientific_notation)

# Remove any columns with all NaN values (fully empty columns)
df_clean = df_numeric.dropna(axis=1, how="all")

print(f"\nProcessed data shape: {df_clean.shape}")
print(f"Number of valid channels: {df_clean.shape[1]}")

# Display basic statistics
print(f"\nBasic Statistics (first 10 channels):")
try:
    print(df_clean.iloc[:, :11].describe())
except ValueError as e:
    print("ValueError during DataFrame.describe():")
    print(
        "Cannot describe a DataFrame without columns.\n"
        "Check if data columns are empty after processing. "
        "Please verify the CSV format and ensure it contains numeric data."
    )
    raise

# Display the first few values
print(f"\nFirst 10 values:")
print(df_clean.iloc[0, :10].values)

# For harmonic analysis, we'll need to identify which channels correspond to which measurements
print(f"\nData loaded successfully!")
print(f"Total channels available: {df_clean.shape[1]}")
print("Note: Channel mapping to specific measurements (Vab, Vbc, Vca, Ia, Ib, Ic) needs to be determined based on PQ3198 configuration.")

Loading HIOKI PQ3198 Power Quality Data...
File path: HIOKI PQ Power Spectrum Sample Datasets/Sample_PQ3198/PQ3198_Dip/CSV/19072900_Dip.csv


/tmp/ipykernel_61113/4065646680.py:11: DtypeWarning: Columns (1970,4116,4176,4179,4181,4182,4185,4187) have mixed types. Specify dtype option on import or set low_memory=False.
  df_raw = pd.read_csv(csv_path, header=0)
/tmp/ipykernel_61113/4065646680.py:43: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df_numeric = df_raw.applymap(parse_scientific_notation)


Raw data shape: (1440, 4196)
Number of columns: 4196

First 20 column names from the CSV:
['Date', 'Time', 'Status1', 'Status2', 'U12_rms1/2_MAX', 'U12_rms1/2_MIN', 'U23_rms1/2_MAX', 'U23_rms1/2_MIN', 'U31_rms1/2_MAX', 'U31_rms1/2_MIN', 'I1_rms1/2_MAX', 'I1_rms1/2_MIN', 'I2_rms1/2_MAX', 'I2_rms1/2_MIN', 'I3_rms1/2_MAX', 'I3_rms1/2_MIN', 'Freq_wav_MAX', 'Freq_wav_MIN', 'U12_rms_MAX', 'U12_rms_AVG']

First row values from the CSV:
['7/29/2019' '12:01:00 AM' np.int64(800000) np.int64(0) np.float64(201.33)
 np.float64(200.14) np.float64(201.26) np.float64(200.06)
 np.float64(200.89) np.float64(199.6) np.float64(0.31) np.float64(0.094)
 np.float64(0.407) np.float64(0.163) np.float64(0.293) np.float64(0.073)
 np.float64(60.145) np.float64(60.057) np.float64(201.2)
 np.float64(200.76)]

Processed data shape: (1440, 4176)
Number of valid channels: 4176

Basic Statistics (first 10 channels):
            Status1  Status2  U12_rms1/2_MAX  U12_rms1/2_MIN  U23_rms1/2_MAX  \
count  1.440000e+03   14

In [10]:
# Analyze the header structure and create channel mapping for 3P3W3M configuration
print("=== HIOKI PQ3198 Data Structure Analysis ===")
print(f"Total data points: {len(parsed_values)}")

# Reference the Hioki PQ3198 module header information for guidance
# Typical PQ3198 harmonic analysis includes:
# - Fundamental frequency components
# - Harmonic components (up to 50th harmonic typically)
# - Phase angles
# - THD values

# Analyze the data distribution
valid_data = [val for val in parsed_values if not np.isnan(val)]
print(f"Valid data points: {len(valid_data)}")
if valid_data:
    print(f"Data range: {min(valid_data):.3f} to {max(valid_data):.3f}")
else:
    print("Data range: No valid data available (empty sequence)")

# Look for patterns in the data
# Values around 100.00E+00 might indicate percentage values (100%)
# Values around 0.12E+00 might indicate harmonic magnitudes
# Values with large ranges might indicate phase angles

# Identify potential channel groups based on data patterns
print("\n=== Data Pattern Analysis ===")

# Group data by magnitude ranges
small_values = [val for val in valid_data if abs(val) < 1.0]
medium_values = [val for val in valid_data if 1.0 <= abs(val) < 50.0]
large_values = [val for val in valid_data if abs(val) >= 50.0]

print(f"Small values (< 1.0): {len(small_values)} points")
print(f"Medium values (1.0-50.0): {len(medium_values)} points") 
print(f"Large values (>= 50.0): {len(large_values)} points")

# Look for 100% values (likely fundamental components)
fundamental_values = [val for val in valid_data if abs(val - 100.0) < 0.1]
print(f"Values near 100% (fundamental): {len(fundamental_values)} points")

# Create a preliminary channel mapping for 3P3W3M
# This is based on typical PQ3198 output structure and documentation header reference
print("\n=== Preliminary Channel Mapping for 3P3W3M (Based on PQ3198 Module Header) ===")

# For 3P3W3M configuration, we typically have:
# - 3 line-to-line voltages: Vab, Vbc, Vca
# - 3 line currents: Ia, Ib, Ic
# - Each measurement has: magnitude, phase angle, THD
# - Plus harmonic components for each

# Let's create a structured view of the data
n_channels = 6  # 3 voltages + 3 currents
n_harmonics = 50  # Typical harmonic analysis up to 50th harmonic

print(f"Assuming {n_channels} main channels (3 voltages + 3 currents)")
print(f"With up to {n_harmonics} harmonic components each")

# Create a more structured DataFrame for analysis
# This is a simplified mapping - actual structure may vary depending on Hioki PQ3198 output format
channel_mapping = {
    'Vab_mag': 'Line-to-line voltage AB magnitude',
    'Vab_phase': 'Line-to-line voltage AB phase angle', 
    'Vab_thd': 'Line-to-line voltage AB THD',
    'Vbc_mag': 'Line-to-line voltage BC magnitude',
    'Vbc_phase': 'Line-to-line voltage BC phase angle',
    'Vbc_thd': 'Line-to-line voltage BC THD', 
    'Vca_mag': 'Line-to-line voltage CA magnitude',
    'Vca_phase': 'Line-to-line voltage CA phase angle',
    'Vca_thd': 'Line-to-line voltage CA THD',
    'Ia_mag': 'Line current A magnitude',
    'Ia_phase': 'Line current A phase angle',
    'Ia_thd': 'Line current A THD',
    'Ib_mag': 'Line current B magnitude', 
    'Ib_phase': 'Line current B phase angle',
    'Ib_thd': 'Line current B THD',
    'Ic_mag': 'Line current C magnitude',
    'Ic_phase': 'Line current C phase angle',
    'Ic_thd': 'Line current C THD'
}

print(f"Created channel mapping for {len(channel_mapping)} primary measurements")
print("Note: This is a preliminary mapping and references Hioki PQ3198 module header documentation. Actual PQ3198 output structure may differ.")
print("Further analysis needed to map specific data points to measurements.")


=== HIOKI PQ3198 Data Structure Analysis ===


NameError: name 'parsed_values' is not defined

In [ ]:
# Build Analysis Scope for 3P3W3M Configuration
print("=== Building Analysis Scope for 3P3W3M Configuration ===")

# For 3P3W3M (3-Phase, 3-Wire, 3-Meter) configuration:
# - Measures line-to-line voltages: Vab, Vbc, Vca
# - Measures line currents: Ia, Ib, Ic  
# - No neutral measurements available
# - No line-to-neutral voltages directly measured

# Create analysis scope structure
analysis_scope = {
    'configuration': '3P3W3M',
    'description': '3-Phase, 3-Wire, 3-Meter configuration',
    'measurements': {
        'voltages': {
            'Vab': {
                'type': 'Line-to-line voltage',
                'description': 'Voltage between phases A and B',
                'measurement_type': 'RMS magnitude, phase angle, THD, harmonics'
            },
            'Vbc': {
                'type': 'Line-to-line voltage', 
                'description': 'Voltage between phases B and C',
                'measurement_type': 'RMS magnitude, phase angle, THD, harmonics'
            },
            'Vca': {
                'type': 'Line-to-line voltage',
                'description': 'Voltage between phases C and A', 
                'measurement_type': 'RMS magnitude, phase angle, THD, harmonics'
            }
        },
        'currents': {
            'Ia': {
                'type': 'Line current',
                'description': 'Current in phase A',
                'measurement_type': 'RMS magnitude, phase angle, THD, harmonics'
            },
            'Ib': {
                'type': 'Line current',
                'description': 'Current in phase B', 
                'measurement_type': 'RMS magnitude, phase angle, THD, harmonics'
            },
            'Ic': {
                'type': 'Line current',
                'description': 'Current in phase C',
                'measurement_type': 'RMS magnitude, phase angle, THD, harmonics'
            }
        }
    },
    'derived_quantities': {
        'power': {
            'active_power': 'P = Vab*Ia*cos(θab-θa) + Vbc*Ib*cos(θbc-θb) + Vca*Ic*cos(θca-θc)',
            'reactive_power': 'Q = Vab*Ia*sin(θab-θa) + Vbc*Ib*sin(θbc-θb) + Vca*Ic*sin(θca-θc)',
            'apparent_power': 'S = sqrt(P² + Q²)'
        },
        'power_factor': 'PF = P/S',
        'frequency': 'Fundamental frequency from voltage measurements',
        'thd': 'Total Harmonic Distortion for each voltage and current'
    },
    'analysis_capabilities': [
        'Harmonic analysis up to 50th harmonic',
        'THD calculation for voltages and currents',
        'Power quality assessment',
        'Frequency analysis',
        'Phase angle relationships',
        'Power calculations (active, reactive, apparent)',
        'Power factor analysis'
    ]
}

# Display the analysis scope
print("Analysis Scope Configuration:")
print(f"Configuration: {analysis_scope['configuration']}")
print(f"Description: {analysis_scope['description']}")

print("\nAvailable Measurements:")
for category, measurements in analysis_scope['measurements'].items():
    print(f"\n{category.upper()}:")
    for name, details in measurements.items():
        print(f"  {name}: {details['description']}")
        print(f"    Type: {details['measurement_type']}")

print("\nDerived Quantities:")
for quantity, formula in analysis_scope['derived_quantities'].items():
    if isinstance(formula, str):
        print(f"  {quantity}: {formula}")
    else:
        print(f"  {quantity}:")
        for sub_quantity, sub_formula in formula.items():
            print(f"    {sub_quantity}: {sub_formula}")

print("\nAnalysis Capabilities:")
for capability in analysis_scope['analysis_capabilities']:
    print(f"  • {capability}")

# Create a summary for the analysis
print(f"\n=== Analysis Summary ===")
print(f"Data source: HIOKI PQ3198 Power Quality Analyzer")
print(f"File: {csv_path}")
print(f"Configuration: {analysis_scope['configuration']}")
print(f"Total data points: {len(valid_data)}")
print(f"Primary measurements: {len(analysis_scope['measurements']['voltages']) + len(analysis_scope['measurements']['currents'])}")
print("Ready for harmonic analysis and power quality assessment.")


In [ ]:
# Create organized data structure for better analysis

# Helper: Print all the columns for inspection (uncomment if needed)
# print(df_clean.columns.tolist())

# Attempt to find matching columns using best-effort flexible matching
import difflib

def find_best_columns(df_columns, queries):
    """
    Try to match a list of 'queries' to the DataFrame columns in a
    case-insensitive and whitespace-insensitive way.
    Returns a dict mapping the query to the matched DataFrame column.
    """
    column_map = {}
    lower_cols = {col.strip().lower(): col for col in df_columns}
    for query in queries:
        key = query.strip().lower()
        if key in lower_cols:
            column_map[query] = lower_cols[key]
        else:
            # Use difflib to find closest match
            matches = difflib.get_close_matches(key, lower_cols.keys(), n=1, cutoff=0.6)
            if matches:
                column_map[query] = lower_cols[matches[0]]
            else:
                column_map[query] = None  # Not found
    return column_map

# Define the "desired" column names (as written in code)
desired_columns = {
    "voltage_columns": ["Vrms ph-n AN Avg", "Vrms ph-n BN Avg", "Vrms ph-n CN Avg"],
    "current_columns": ["Current A Avg", "Current B Avg", "Current C Avg"],
    "phase_columns": ["Current Phi AN Avg", "Current Phi BN Avg", "Current Phi CN Avg"],
    "cos_phi_columns": ["Cos Phi AN Avg", "Cos Phi BN Avg", "Cos Phi CN Avg"],
    "power_columns": ["Active Power Total Avg", "Apparent Power Total Avg", "Reactive Power Total Avg"],
    "neutral_voltage": ["Vrms ph-n NG Avg"],
    "total_power_factor": ["Cos Phi Total Avg"],
}

# Try to find columns in df_clean matching these "logical" column names
mapped_columns = {}
for key, queries in desired_columns.items():
    match_dict = find_best_columns(df_clean.columns, queries)
    # Only include matches that were actually found
    found = [v for v in match_dict.values() if v is not None]
    if len(found) != len(queries):
        # Warn about unmatched columns (optional)
        for q, v in match_dict.items():
            if v is None:
                print(f"Warning: Could not find column matching '{q}' in df_clean.columns.")
    mapped_columns[key] = found

# Assign the mapped columns to variable names as before
voltage_columns = mapped_columns['voltage_columns']
current_columns = mapped_columns['current_columns']
phase_columns = mapped_columns['phase_columns']
cos_phi_columns = mapped_columns['cos_phi_columns']
power_columns = mapped_columns['power_columns']

# Create a comprehensive data dictionary for easy reference (using the found columns)
data_info = {
    'voltage_phases': voltage_columns,
    'current_phases': current_columns,
    'phase_angles': phase_columns,
    'power_factors': cos_phi_columns,
    'power_quantities': power_columns,
    'neutral_voltage': mapped_columns['neutral_voltage'],
    'total_power_factor': mapped_columns['total_power_factor']
}

print("DataFrame Structure:")
print("=" * 50)
for category, columns in data_info.items():
    print(f"{category.replace('_', ' ').title()}: {columns}")

if not (voltage_columns and current_columns):
    print("\nError: Could not find all the required voltage or current columns. Please check the CSV header names.")

else:
    # Create a summary DataFrame for quick analysis
    summary_stats = pd.DataFrame({
        'Mean': df_clean[voltage_columns + current_columns].mean(),
        'Std': df_clean[voltage_columns + current_columns].std(),
        'Min': df_clean[voltage_columns + current_columns].min(),
        'Max': df_clean[voltage_columns + current_columns].max()
    })

    print(f"\nSummary Statistics for Voltage and Current:")
    print(summary_stats.round(2))


## Power Characteristics 
The `power.characteristics` submodule is used to compute quantities of interest from voltage and current time-series. In this example, we will calculate active AC power and instantneous frequency.

### Active AC Power
To compute the active AC power, the user must provide a `power_factor` for the efficency of the device. 


In [ ]:
# Extract voltage and current data using organized column structure
voltage = df_clean[voltage_columns].copy()
current = df_clean[current_columns].copy()

# Rename columns for better readability
voltage.columns = ['Phase_A_Voltage', 'Phase_B_Voltage', 'Phase_C_Voltage']
current.columns = ['Phase_A_Current', 'Phase_B_Current', 'Phase_C_Current']

print("Voltage and Current DataFrames created:")
print(f"Voltage shape: {voltage.shape}")
print(f"Current shape: {current.shape}")
print(f"\nVoltage columns: {list(voltage.columns)}")
print(f"Current columns: {list(current.columns)}")

# Set the power factor for the system
# Option 1: Use a fixed power factor
# power_factor = 0.96

# Option 2: Calculate power factor as the mean of (Active Power Total Avg / Apparent Power Total Avg)
power_factor = (df_clean['Active Power Total Avg'] / df_clean['Apparent Power Total Avg']).mean()
print(f"Mean power factor used: {power_factor:.3f}")

# Compute the instantaneous AC power in watts
ac_power = power.characteristics.ac_power_three_phase(voltage, current, power_factor)

# Display the result with enhanced visualization
fig, ax = plt.subplots(figsize=(15, 6))
ac_power.Power.plot(ax=ax, title="AC Power - Three Phase", color='blue', linewidth=1)
ax.set_xlabel("DateTime")
ax.set_ylabel("Power (W)")
ax.grid(True, alpha=0.3)
ax.set_ylim(0, 2000000)  # Adjusted y-axis limit based on data range
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

# Display power statistics
print(f"\nAC Power Statistics:")
print(f"Mean Power: {ac_power.Power.mean():.0f} W")
print(f"Max Power: {ac_power.Power.max():.0f} W")
print(f"Min Power: {ac_power.Power.min():.0f} W")
print(f"Std Power: {ac_power.Power.std():.0f} W")

In [ ]:
# Comprehensive Data Quality Assessment
print("=" * 60)
print("DATA QUALITY ASSESSMENT")
print("=" * 60)

# 1. Data completeness
print("1. Data Completeness:")
print(f"   Total records: {len(df_clean)}")
print(f"   Missing values: {df_clean.isnull().sum().sum()}")
print(f"   Data completeness: {((len(df_clean) - df_clean.isnull().sum().sum()) / (len(df_clean) * len(df_clean.columns)) * 100):.1f}%")

# 2. Time series continuity
time_diffs = df_clean.index.to_series().diff().dt.total_seconds()
print(f"\n2. Time Series Analysis:")
print(f"   Time span: {df_clean.index.max() - df_clean.index.min()}")
print(f"   Average sampling interval: {time_diffs.mean():.1f} seconds")
print(f"   Sampling frequency: {1/time_diffs.mean():.1f} Hz")
print(f"   Time gaps detected: {(time_diffs > time_diffs.median() * 2).sum()}")

# 3. Electrical parameter validation
print(f"\n3. Electrical Parameter Validation:")
print(f"   Voltage range: {df_clean[voltage_columns].min().min():.1f} - {df_clean[voltage_columns].max().max():.1f} V")
print(f"   Current range: {df_clean[current_columns].min().min():.1f} - {df_clean[current_columns].max().max():.1f} A")
print(f"   Power range: {df_clean['Active Power Total Avg'].min():.0f} - {df_clean['Active Power Total Avg'].max():.0f} W")

# 4. Phase balance analysis
voltage_balance = df_clean[voltage_columns].std(axis=1).mean()
current_balance = df_clean[current_columns].std(axis=1).mean()
print(f"\n4. Phase Balance Analysis:")
print(f"   Average voltage unbalance: {voltage_balance:.2f} V")
print(f"   Average current unbalance: {current_balance:.2f} A")

# 5. Power factor analysis
pf_analysis = df_clean[cos_phi_columns].describe()
print(f"\n5. Power Factor Analysis:")
print(f"   Phase A PF: {pf_analysis['Cos Phi AN Avg']['mean']:.3f} ± {pf_analysis['Cos Phi AN Avg']['std']:.3f}")
print(f"   Phase B PF: {pf_analysis['Cos Phi BN Avg']['mean']:.3f} ± {pf_analysis['Cos Phi BN Avg']['std']:.3f}")
print(f"   Phase C PF: {pf_analysis['Cos Phi CN Avg']['mean']:.3f} ± {pf_analysis['Cos Phi CN Avg']['std']:.3f}")

print("\n" + "=" * 60)


### Instantaneous Frequency

Calculate the instantaneous frequency of the voltage time-series using the 3 phase voltage measurements. The time-varying instantaneous frequency is a required metric for power quality assessments. 

In [ ]:
# Compute the instantaneous frequency
inst_freq = power.characteristics.instantaneous_frequency(voltage)
display(voltage.head())

# Display the result
inst_freq.plot(figsize=(15, 5), ylim=(0, 100), title="Instantaneous Frequency").set(
    xlabel="Time [s]", ylabel="Frequency [Hz]"
)
inst_freq.head()

## Power Quality
The `power.quality` module can be used to compute current or voltage harmonics and current distortions following IEC TS 62600-30 and IEC TS 61000-4-7. Harmonics and harmonic distortion are required as part of a power quality assessment and characterize the stability of the produced power. 

In [ ]:
# Set the nominal sampling frequency
sample_freq = 50000  # [Hz]

# Set the frequency of the grid the device would be conected to
grid_freq = 60  # [Hz]

# Set the rated current of the device
rated_current = 600  # [Amps]

# Calculate the harmonics
harmonics = power.quality.harmonics(current, sample_freq, grid_freq)

# Plot the results
harmonics.plot(figsize=(15, 5), xlim=(0, 900), title="Current Harmonics").set(
    ylabel="Harmonic Amplitude", xlabel="Frequency [Hz]"
);

### Harmonic Subgroups
The harmonic subgroups calculations are based on IEC TS 62600-30. We can calculate them using our grid frequency and harmonics.

In [ ]:
# Calculate Harmonic Subgroups
h_s = power.quality.harmonic_subgroups(harmonics, grid_freq)

# Add harmonic order as a column based on index/frequency and grid_freq
h_s = h_s.copy()
h_s['Harmonic Order'] = h_s.index / grid_freq

# Display the results with harmonic order
h_s.head(50)

In [ ]:
# Global FFT Parameters for all analysis
# These parameters will be used throughout the FFT analysis sections
nfft = 1024  # Number of FFT points
fs = 50000   # Sampling frequency [Hz] - consistent with harmonics analysis

print(f"FFT Parameters set:")
print(f"  FFT points (nfft): {nfft}")
print(f"  Sampling frequency (fs): {fs} Hz")
print(f"  Frequency resolution: {fs/nfft:.1f} Hz")


### Total Harmonic Current Distortion

Compute the THCD from harmonic subgroups and rated current for the device

## MHKiT Dolfyn FFT Analysis

The `mhkit.dolfyn.tools.fft` module provides advanced spectral analysis functions for power quality assessment. We'll demonstrate three key functions:

1. **`fft_frequency()`** - Computes frequency vector for FFT analysis
2. **`cpsd_quasisync_1D()`** - Cross power spectral density for two signals
3. **`psd_1D()`** - Power spectral density for single signal analysis

These functions are particularly useful for analyzing the spectral characteristics of voltage and current signals in power systems.


### 1. FFT Frequency Vector

First, let's compute the frequency vector for our FFT analysis using the sample frequency from our power data.


In [ ]:
# Prepare data for PSD analysis - take a subset for demonstration
# Convert to numpy arrays and remove NaN values
# Use the renamed column names from our organized DataFrame structure
voltage_a = voltage["Phase_A_Voltage"].dropna().values
voltage_b = voltage["Phase_B_Voltage"].dropna().values
voltage_c = voltage["Phase_C_Voltage"].dropna().values
current_a = current["Phase_A_Current"].dropna().values
current_b = current["Phase_B_Current"].dropna().values
current_c = current["Phase_C_Current"].dropna().values

print(f"Data lengths - Voltage A: {len(voltage_a)}, Current A: {len(current_a)}")

# Set FFT parameters for PSD analysis
nfft = 1024  # Number of FFT points
fs = 50000   # Sampling frequency [Hz] - using the same value from harmonics analysis

# Compute PSD for voltage signals
psd_voltage_a = psd_1D(voltage_a, nfft, fs, window='hann')
psd_voltage_b = psd_1D(voltage_b, nfft, fs, window='hann')
psd_voltage_c = psd_1D(voltage_c, nfft, fs, window='hann')

# Compute PSD for current signals
psd_current_a = psd_1D(current_a, nfft, fs, window='hann')
psd_current_b = psd_1D(current_b, nfft, fs, window='hann')
psd_current_c = psd_1D(current_c, nfft, fs, window='hann')

print(f"PSD array lengths: {len(psd_voltage_a)}")
print(f"PSD voltage A - Max: {np.max(psd_voltage_a):.2e}, Min: {np.min(psd_voltage_a):.2e}")
print(f"PSD current A - Max: {np.max(psd_current_a):.2e}, Min: {np.min(psd_current_a):.2e}")


In [ ]:
# CORRECTED PSD Analysis Cell - Use this instead of any old cells
# This cell uses the correct renamed column names and will override any old cells

# Prepare data for PSD analysis - take a subset for demonstration
# Convert to numpy arrays and remove NaN values
# Use the renamed column names from our organized DataFrame structure
voltage_a = voltage["Phase_A_Voltage"].dropna().values
voltage_b = voltage["Phase_B_Voltage"].dropna().values
voltage_c = voltage["Phase_C_Voltage"].dropna().values
current_a = current["Phase_A_Current"].dropna().values
current_b = current["Phase_B_Current"].dropna().values
current_c = current["Phase_C_Current"].dropna().values

print(f"Data lengths - Voltage A: {len(voltage_a)}, Current A: {len(current_a)}")

# Set FFT parameters for PSD analysis
nfft = 1024  # Number of FFT points
fs = 50000   # Sampling frequency [Hz] - using the same value from harmonics analysis

# Compute PSD for voltage signals
psd_voltage_a = psd_1D(voltage_a, nfft, fs, window='hann')
psd_voltage_b = psd_1D(voltage_b, nfft, fs, window='hann')
psd_voltage_c = psd_1D(voltage_c, nfft, fs, window='hann')

# Compute PSD for current signals
psd_current_a = psd_1D(current_a, nfft, fs, window='hann')
psd_current_b = psd_1D(current_b, nfft, fs, window='hann')
psd_current_c = psd_1D(current_c, nfft, fs, window='hann')

print(f"PSD array lengths: {len(psd_voltage_a)}")
print(f"PSD voltage A - Max: {np.max(psd_voltage_a):.2e}, Min: {np.min(psd_voltage_a):.2e}")
print(f"PSD current A - Max: {np.max(psd_current_a):.2e}, Min: {np.min(psd_current_a):.2e}")


In [ ]:
# Set FFT parameters
nfft = 1024  # Number of FFT points
fs = sample_freq  # Sampling frequency

# Compute frequency vector
freq_vector = fft_frequency(nfft, fs, full=False)
print(f"Frequency vector length: {len(freq_vector)}")
print(f"Frequency range: {freq_vector[0]:.2f} Hz to {freq_vector[-1]:.2f} Hz")
print(f"Frequency resolution: {freq_vector[1] - freq_vector[0]:.2f} Hz")

# Display first 10 frequencies
print(f"\nFirst 10 frequencies: {freq_vector[:10]}")

# --- Draw FFT chart for all three phases of current on the same plot ---
from scipy.fft import fft
import matplotlib.pyplot as plt
import numpy as np

# Obtain each phase's current; fallback to array of ones if not available
if 'current_a' in locals():
    signal_a = current_a
else:
    try:
        signal_a = current["Current A Avg"].dropna().values
    except:
        signal_a = np.ones(nfft)

if 'current_b' in locals():
    signal_b = current_b
else:
    try:
        signal_b = current["Current B Avg"].dropna().values
    except:
        signal_b = np.ones(nfft)

if 'current_c' in locals():
    signal_c = current_c
else:
    try:
        signal_c = current["Current C Avg"].dropna().values
    except:
        signal_c = np.ones(nfft)

# Make sure each is long enough for FFT
def pad_signal(sig, length):
    if len(sig) >= length:
        return sig[:length]
    else:
        return np.pad(sig, (0, length - len(sig)), 'constant')

signal_a = pad_signal(signal_a, nfft)
signal_b = pad_signal(signal_b, nfft)
signal_c = pad_signal(signal_c, nfft)

fft_a = fft(signal_a, n=nfft)
fft_b = fft(signal_b, n=nfft)
fft_c = fft(signal_c, n=nfft)

# Calculate amplitude to show the actual value of current (not normalized)
# The FFT output (for real-valued signals) must be scaled as follows:
#   - Right-side bins (except DC and Nyquist) are doubled, to get real amplitude.
#   - Amplitudes are magnitude values, divided by nfft.
def fft_amplitude(fft_result, nfft):
    amp = np.abs(fft_result) / nfft
    # Double all except DC (0) and Nyquist (nfft/2 if even)
    if nfft % 2 == 0:  # even
        amp[1:nfft//2] = amp[1:nfft//2] * 2
    else:  # odd
        amp[1:(nfft+1)//2] = amp[1:(nfft+1)//2] * 2
    return amp[:len(freq_vector)]

fft_amp_a = fft_amplitude(fft_a, nfft)
fft_amp_b = fft_amplitude(fft_b, nfft)
fft_amp_c = fft_amplitude(fft_c, nfft)

plt.figure(figsize=(12, 5))
plt.plot(freq_vector, fft_amp_a, label='Current A', color='b')
plt.plot(freq_vector, fft_amp_b, label='Current B', color='g')
plt.plot(freq_vector, fft_amp_c, label='Current C', color='r')
plt.title('FFT of AC Current Harmonics (A, B, C)')
plt.xlabel('Frequency (Hz)')
plt.ylabel('Current Amplitude (A) Ratio')
plt.grid(True, which='both')
plt.legend()
plt.tight_layout()
plt.show()


### 2. Power Spectral Density (PSD) Analysis

Now let's compute the power spectral density for each phase of voltage and current to analyze their spectral characteristics.


In [ ]:
# Compute CPSD between voltage and current for each phase
# Note: cpsd_quasisync_1D can handle signals of different lengths

# Ensure FFT parameters are available (in case this cell is run independently)
if 'nfft' not in locals():
    nfft = 1024  # Number of FFT points
if 'fs' not in locals():
    fs = 50000   # Sampling frequency [Hz]

cpsd_va_ia = cpsd_quasisync_1D(voltage_a, current_a, nfft, fs, window='hann')
cpsd_vb_ib = cpsd_quasisync_1D(voltage_b, current_b, nfft, fs, window='hann')
cpsd_vc_ic = cpsd_quasisync_1D(voltage_c, current_c, nfft, fs, window='hann')

# Also compute CPSD between different phases for comparison
cpsd_va_vb = cpsd_quasisync_1D(voltage_a, voltage_b, nfft, fs, window='hann')
cpsd_ia_ib = cpsd_quasisync_1D(current_a, current_b, nfft, fs, window='hann')

print(f"CPSD array lengths: {len(cpsd_va_ia)}")
print(f"CPSD V-A vs I-A - Max: {np.max(np.abs(cpsd_va_ia)):.2e}, Min: {np.min(np.abs(cpsd_va_ia)):.2e}")
print(f"CPSD V-A vs V-B - Max: {np.max(np.abs(cpsd_va_vb)):.2e}, Min: {np.min(np.abs(cpsd_va_vb)):.2e}")


In [ ]:
# Prepare data for PSD analysis - take a subset for demonstration
# Convert to numpy arrays and remove NaN values
# Use the renamed column names from our organized DataFrame structure
voltage_a = voltage["Phase_A_Voltage"].dropna().values
voltage_b = voltage["Phase_B_Voltage"].dropna().values
voltage_c = voltage["Phase_C_Voltage"].dropna().values
current_a = current["Phase_A_Current"].dropna().values
current_b = current["Phase_B_Current"].dropna().values
current_c = current["Phase_C_Current"].dropna().values

print(f"Data lengths - Voltage A: {len(voltage_a)}, Current A: {len(current_a)}")

# Compute PSD for voltage signals
psd_voltage_a = psd_1D(voltage_a, nfft, fs, window='hann')
psd_voltage_b = psd_1D(voltage_b, nfft, fs, window='hann')
psd_voltage_c = psd_1D(voltage_c, nfft, fs, window='hann')

# Compute PSD for current signals
psd_current_a = psd_1D(current_a, nfft, fs, window='hann')
psd_current_b = psd_1D(current_b, nfft, fs, window='hann')
psd_current_c = psd_1D(current_c, nfft, fs, window='hann')

print(f"PSD array lengths: {len(psd_voltage_a)}")
print(f"PSD voltage A - Max: {np.max(psd_voltage_a):.2e}, Min: {np.min(psd_voltage_a):.2e}")
print(f"PSD current A - Max: {np.max(psd_current_a):.2e}, Min: {np.min(psd_current_a):.2e}")


In [ ]:
# CORRECTED CPSD Analysis Cell - Use this instead of any old cells
# This cell uses the correct variable names and will override any old cells

# Compute CPSD between voltage and current for each phase
# Note: cpsd_quasisync_1D can handle signals of different lengths

# Ensure FFT parameters are available (in case this cell is run independently)
if 'nfft' not in locals():
    nfft = 1024  # Number of FFT points
if 'fs' not in locals():
    fs = 50000   # Sampling frequency [Hz]

cpsd_va_ia = cpsd_quasisync_1D(voltage_a, current_a, nfft, fs, window='hann')
cpsd_vb_ib = cpsd_quasisync_1D(voltage_b, current_b, nfft, fs, window='hann')
cpsd_vc_ic = cpsd_quasisync_1D(voltage_c, current_c, nfft, fs, window='hann')

# Also compute CPSD between different phases for comparison
cpsd_va_vb = cpsd_quasisync_1D(voltage_a, voltage_b, nfft, fs, window='hann')
cpsd_ia_ib = cpsd_quasisync_1D(current_a, current_b, nfft, fs, window='hann')

print(f"CPSD array lengths: {len(cpsd_va_ia)}")
print(f"CPSD V-A vs I-A - Max: {np.max(np.abs(cpsd_va_ia)):.2e}, Min: {np.min(np.abs(cpsd_va_ia)):.2e}")
print(f"CPSD V-A vs V-B - Max: {np.max(np.abs(cpsd_va_vb)):.2e}, Min: {np.min(np.abs(cpsd_va_vb)):.2e}")


In [ ]:
# Plot PSD results
fig, axes = plt.subplots(2, 3, figsize=(18, 10))

# Plot voltage PSDs
axes[0, 0].semilogy(freq_vector, psd_voltage_a)
axes[0, 0].set_title('Voltage A PSD')
axes[0, 0].set_xlabel('Frequency (Hz)')
axes[0, 0].set_ylabel('PSD (V²/Hz)')
axes[0, 0].grid(True)
axes[0, 0].set_xlim(0, 1000)  # Focus on lower frequencies

axes[0, 1].semilogy(freq_vector, psd_voltage_b)
axes[0, 1].set_title('Voltage B PSD')
axes[0, 1].set_xlabel('Frequency (Hz)')
axes[0, 1].set_ylabel('PSD (V²/Hz)')
axes[0, 1].grid(True)
axes[0, 1].set_xlim(0, 1000)

axes[0, 2].semilogy(freq_vector, psd_voltage_c)
axes[0, 2].set_title('Voltage C PSD')
axes[0, 2].set_xlabel('Frequency (Hz)')
axes[0, 2].set_ylabel('PSD (V²/Hz)')
axes[0, 2].grid(True)
axes[0, 2].set_xlim(0, 1000)

# Plot current PSDs
axes[1, 0].semilogy(freq_vector, psd_current_a)
axes[1, 0].set_title('Current A PSD')
axes[1, 0].set_xlabel('Frequency (Hz)')
axes[1, 0].set_ylabel('PSD (A²/Hz)')
axes[1, 0].grid(True)
axes[1, 0].set_xlim(0, 1000)

axes[1, 1].semilogy(freq_vector, psd_current_b)
axes[1, 1].set_title('Current B PSD')
axes[1, 1].set_xlabel('Frequency (Hz)')
axes[1, 1].set_ylabel('PSD (A²/Hz)')
axes[1, 1].grid(True)
axes[1, 1].set_xlim(0, 1000)

axes[1, 2].semilogy(freq_vector, psd_current_c)
axes[1, 2].set_title('Current C PSD')
axes[1, 2].set_xlabel('Frequency (Hz)')
axes[1, 2].set_ylabel('PSD (A²/Hz)')
axes[1, 2].grid(True)
axes[1, 2].set_xlim(0, 1000)

plt.tight_layout()
plt.show()


### 3. Cross Power Spectral Density (CPSD) Analysis

Now let's compute the cross power spectral density between voltage and current signals to analyze their relationship in the frequency domain. This is useful for understanding power quality and phase relationships.


In [ ]:
# Compute CPSD between voltage and current for each phase
# Note: cpsd_quasisync_1D can handle signals of different lengths
cpsd_va_ia = cpsd_quasisync_1D(voltage_a, current_a, nfft, fs, window='hann')
cpsd_vb_ib = cpsd_quasisync_1D(voltage_b, current_b, nfft, fs, window='hann')
cpsd_vc_ic = cpsd_quasisync_1D(voltage_c, current_c, nfft, fs, window='hann')

# Also compute CPSD between different phases for comparison
cpsd_va_vb = cpsd_quasisync_1D(voltage_a, voltage_b, nfft, fs, window='hann')
cpsd_ia_ib = cpsd_quasisync_1D(current_a, current_b, nfft, fs, window='hann')

print(f"CPSD array lengths: {len(cpsd_va_ia)}")
print(f"CPSD V-A vs I-A - Max: {np.max(np.abs(cpsd_va_ia)):.2e}, Min: {np.min(np.abs(cpsd_va_ia)):.2e}")
print(f"CPSD V-A vs V-B - Max: {np.max(np.abs(cpsd_va_vb)):.2e}, Min: {np.min(np.abs(cpsd_va_vb)):.2e}")


In [ ]:
# Plot CPSD results
fig, axes = plt.subplots(2, 3, figsize=(18, 10))

# Plot voltage-current CPSDs (magnitude)
axes[0, 0].semilogy(freq_vector, np.abs(cpsd_va_ia))
axes[0, 0].set_title('CPSD: Voltage A vs Current A (Magnitude)')
axes[0, 0].set_xlabel('Frequency (Hz)')
axes[0, 0].set_ylabel('|CPSD| (V·A/Hz)')
axes[0, 0].grid(True)
axes[0, 0].set_xlim(0, 1000)

axes[0, 1].semilogy(freq_vector, np.abs(cpsd_vb_ib))
axes[0, 1].set_title('CPSD: Voltage B vs Current B (Magnitude)')
axes[0, 1].set_xlabel('Frequency (Hz)')
axes[0, 1].set_ylabel('|CPSD| (V·A/Hz)')
axes[0, 1].grid(True)
axes[0, 1].set_xlim(0, 1000)

axes[0, 2].semilogy(freq_vector, np.abs(cpsd_vc_ic))
axes[0, 2].set_title('CPSD: Voltage C vs Current C (Magnitude)')
axes[0, 2].set_xlabel('Frequency (Hz)')
axes[0, 2].set_ylabel('|CPSD| (V·A/Hz)')
axes[0, 2].grid(True)
axes[0, 2].set_xlim(0, 1000)

# Plot phase relationships
axes[1, 0].semilogy(freq_vector, np.abs(cpsd_va_vb))
axes[1, 0].set_title('CPSD: Voltage A vs Voltage B (Magnitude)')
axes[1, 0].set_xlabel('Frequency (Hz)')
axes[1, 0].set_ylabel('|CPSD| (V²/Hz)')
axes[1, 0].grid(True)
axes[1, 0].set_xlim(0, 1000)

axes[1, 1].semilogy(freq_vector, np.abs(cpsd_ia_ib))
axes[1, 1].set_title('CPSD: Current A vs Current B (Magnitude)')
axes[1, 1].set_xlabel('Frequency (Hz)')
axes[1, 1].set_ylabel('|CPSD| (A²/Hz)')
axes[1, 1].grid(True)
axes[1, 1].set_xlim(0, 1000)

# Plot phase angle for voltage-current relationship
phase_angle = np.angle(cpsd_va_ia)
axes[1, 2].plot(freq_vector, phase_angle * 180 / np.pi)
axes[1, 2].set_title('Phase Angle: Voltage A vs Current A')
axes[1, 2].set_xlabel('Frequency (Hz)')
axes[1, 2].set_ylabel('Phase Angle (degrees)')
axes[1, 2].grid(True)
axes[1, 2].set_xlim(0, 1000)
axes[1, 2].set_ylim(-180, 180)

plt.tight_layout()
plt.show()


### 4. Summary of MHKiT Dolfyn FFT Functions

The three MHKiT dolfyn FFT functions provide comprehensive spectral analysis capabilities:

**`fft_frequency(nfft, fs, full=False)`**
- Computes frequency vector for FFT analysis
- Essential for plotting and interpreting spectral results
- Parameters: number of FFT points, sampling frequency, full spectrum flag

**`psd_1D(a, nfft, fs, window='hann', step=None)`**
- Computes power spectral density of a single signal
- Useful for analyzing individual voltage or current signals
- Provides frequency domain representation of signal power

**`cpsd_quasisync_1D(a, b, nfft, fs, window='hann')`**
- Computes cross power spectral density between two signals
- Handles signals of different lengths (quasi-synchronous)
- Essential for analyzing relationships between voltage and current signals
- Provides both magnitude and phase information

These functions complement the existing MHKiT power module functions and provide advanced spectral analysis capabilities for power quality assessment.


In [ ]:
# Finally we can compute the total harmonic current distortion as a percentage
THCD = power.quality.total_harmonic_current_distortion(h_s)
THCD

In [ ]:
# Multi-Panel Chart: Urms, Irms, and Frequency Analysis
# Create a comprehensive three-panel visualization of the 3-AC Power Meter dataset
import matplotlib.pyplot as plt
import numpy as np
from datetime import datetime

# --- Import the relevant MHKiT function for frequency estimation
from mhkit.power import quality as power_quality

# Set up the figure with three subplots
fig, (ax1, ax2, ax3) = plt.subplots(3, 1, figsize=(15, 12))
fig.suptitle('3-AC Power Meter Dataset Analysis', fontsize=16, fontweight='bold')

# Get the time index for x-axis
time_index = df_clean.index

# ===== TOP PLOT: Urms (Voltage RMS) =====
# Calculate RMS voltage for each phase and average
voltage_rms_a = df_clean['Vrms ph-n AN Avg']
voltage_rms_b = df_clean['Vrms ph-n BN Avg']
voltage_rms_c = df_clean['Vrms ph-n CN Avg']

# Plot all three phases
ax1.plot(time_index, voltage_rms_a, label='Phase A', color='red', linewidth=1.5, alpha=0.8)
ax1.plot(time_index, voltage_rms_b, label='Phase B', color='green', linewidth=1.5, alpha=0.8)
ax1.plot(time_index, voltage_rms_c, label='Phase C', color='blue', linewidth=1.5, alpha=0.8)

# Add average line
voltage_rms_avg = (voltage_rms_a + voltage_rms_b + voltage_rms_c) / 3
ax1.plot(time_index, voltage_rms_avg, label='Average', color='black', linewidth=2, linestyle='--')

ax1.set_title('Urms - Root Mean Square Voltage', fontsize=14, fontweight='bold')
ax1.set_ylabel('Voltage (V)', fontsize=12)
ax1.grid(True, alpha=0.3)
ax1.legend(loc='upper right')
ax1.set_ylim(0, 260)

# ===== MIDDLE PLOT: Irms (Current RMS) =====
# Get current data
current_rms_a = df_clean['Current A Avg']
current_rms_b = df_clean['Current B Avg']
current_rms_c = df_clean['Current C Avg']

# Plot all three phases
ax2.plot(time_index, current_rms_a, label='Phase A', color='red', linewidth=1.5, alpha=0.8)
ax2.plot(time_index, current_rms_b, label='Phase B', color='green', linewidth=1.5, alpha=0.8)
ax2.plot(time_index, current_rms_c, label='Phase C', color='blue', linewidth=1.5, alpha=0.8)

# Add average line
current_rms_avg = (current_rms_a + current_rms_b + current_rms_c) / 3
ax2.plot(time_index, current_rms_avg, label='Average', color='black', linewidth=2, linestyle='--')

ax2.set_title('Irms - Root Mean Square Current', fontsize=14, fontweight='bold')
ax2.set_ylabel('Current (A)', fontsize=12)
ax2.grid(True, alpha=0.3)
ax2.legend(loc='upper right')

# ===== BOTTOM PLOT: Instantaneous Frequency using MHKiT =====
# Try to compute instantaneous frequency using MHKiT (power.quality.frequency)
# The function signature is: power.quality.frequency(voltage, fs), where
# voltage: array-like
# fs: sampling frequency [Hz]

# For this data, assume "Vrms ph-n AN Avg", etc., are average RMS per row;
# If you only have averaged data (not samples), true freq estimation is not possible.
# If you DO have voltage waveform samples per unit time, you can use them for true frequency extraction.
# Here, try with what is in the table—demonstrate how the MHKiT function would be used when waveform data is available.

# Let's try to estimate the sampling frequency from the timestamps:
if isinstance(df_clean.index, pd.DatetimeIndex) and len(df_clean) > 1:
    # Calculate mean sampling period in seconds
    delta_seconds = (df_clean.index[1:] - df_clean.index[:-1]).total_seconds()
    mean_sample_period = np.mean(delta_seconds)
    if mean_sample_period > 0:
        fs_est = 1.0 / mean_sample_period
    else:
        fs_est = 1.0
else:
    fs_est = 1.0  # Fallback if index is not timedelta

# Convert voltage signals to numpy arrays
voltage_signal_a = voltage_rms_a.values
voltage_signal_b = voltage_rms_b.values
voltage_signal_c = voltage_rms_c.values

# Use MHKiT power.quality.frequency function, understanding this is demonstration on RMS not actual waveform
try:
    freq_inst_a = power_quality.frequency(voltage_signal_a, fs_est)
    freq_inst_b = power_quality.frequency(voltage_signal_b, fs_est)
    freq_inst_c = power_quality.frequency(voltage_signal_c, fs_est)
    freq_inst_avg = (freq_inst_a + freq_inst_b + freq_inst_c) / 3
except Exception as e:
    # Fallback to 60Hz array if MHKiT fails (e.g. not enough or not proper data)
    freq_inst_a = np.full_like(voltage_signal_a, 60.0)
    freq_inst_b = np.full_like(voltage_signal_b, 60.0)
    freq_inst_c = np.full_like(voltage_signal_c, 60.0)
    freq_inst_avg = np.full_like(voltage_signal_a, 60.0)
    print(f"Could not compute instantaneous frequency from MHKiT: {e}")

# Plot frequencies, using main time axis
ax3.plot(time_index, freq_inst_a, label='Phase A', color='red', linewidth=1.5, alpha=0.8)
ax3.plot(time_index, freq_inst_b, label='Phase B', color='green', linewidth=1.5, alpha=0.8)
ax3.plot(time_index, freq_inst_c, label='Phase C', color='blue', linewidth=1.5, alpha=0.8)
ax3.plot(time_index, freq_inst_avg, label='Average', color='black', linewidth=2, linestyle='--')

ax3.axhline(y=60, color='orange', linestyle=':', linewidth=2, alpha=0.7, label='60 Hz Reference')
ax3.set_title('Instantaneous Frequency (MHKiT power.quality.frequency)', fontsize=14, fontweight='bold')
ax3.set_ylabel('Frequency (Hz)', fontsize=12)
ax3.set_xlabel('Time', fontsize=12)
ax3.grid(True, alpha=0.3)
ax3.legend(loc='upper right')
ax3.set_ylim(55, 65)

# Format x-axis for all plots
for ax in [ax1, ax2, ax3]:
    ax.tick_params(axis='x', rotation=45)
    ax.xaxis.set_major_formatter(plt.matplotlib.dates.DateFormatter('%H:%M'))

# Adjust layout and display
plt.tight_layout()
plt.subplots_adjust(top=0.93)
plt.show()

# Print summary statistics
print("=" * 60)
print("SUMMARY STATISTICS")
print("=" * 60)
print(f"Voltage RMS (V):")
print(f"  Phase A: {voltage_rms_a.mean():.1f} ± {voltage_rms_a.std():.1f} V")
print(f"  Phase B: {voltage_rms_b.mean():.1f} ± {voltage_rms_b.std():.1f} V")
print(f"  Phase C: {voltage_rms_c.mean():.1f} ± {voltage_rms_c.std():.1f} V")
print(f"  Average: {voltage_rms_avg.mean():.1f} ± {voltage_rms_avg.std():.1f} V")

print(f"\nCurrent RMS (A):")
print(f"  Phase A: {current_rms_a.mean():.1f} ± {current_rms_a.std():.1f} A")
print(f"  Phase B: {current_rms_b.mean():.1f} ± {current_rms_b.std():.1f} A")
print(f"  Phase C: {current_rms_c.mean():.1f} ± {current_rms_c.std():.1f} A")
print(f"  Average: {current_rms_avg.mean():.1f} ± {current_rms_avg.std():.1f} A")

if np.allclose(freq_inst_avg, 60.0):
    print(f"\nInstantaneous Frequency (MHKiT estimated from RMS):")
    print(f"  Could not compute from waveform, used 60 Hz fallback (data averaged, not waveform).")
else:
    print(f"\nInstantaneous Frequency (Hz) (MHKiT power.quality.frequency):")
    print(f"  Phase A: {np.mean(freq_inst_a):.2f} ± {np.std(freq_inst_a):.2f} Hz")
    print(f"  Phase B: {np.mean(freq_inst_b):.2f} ± {np.std(freq_inst_b):.2f} Hz")
    print(f"  Phase C: {np.mean(freq_inst_c):.2f} ± {np.std(freq_inst_c):.2f} Hz")
    print(f"  Average: {np.mean(freq_inst_avg):.2f} ± {np.std(freq_inst_avg):.2f} Hz")


In [ ]:
# Voltage and Current Unbalance Factor Analysis
# Calculate and plot Uunb% and Iunb% for three-phase power system analysis

import matplotlib.pyplot as plt
import numpy as np

# Set up the figure with two subplots
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(15, 10))
fig.suptitle('Three-Phase Unbalance Factor Analysis', fontsize=16, fontweight='bold')

# Get the time index for x-axis
time_index = df_clean.index

# ===== CALCULATE VOLTAGE UNBALANCE FACTOR (Uunb%) =====
# Voltage unbalance factor is calculated as: (Max deviation from average / Average) * 100
voltage_a = df_clean['Vrms ph-n AN Avg']
voltage_b = df_clean['Vrms ph-n BN Avg']
voltage_c = df_clean['Vrms ph-n CN Avg']

# Calculate average voltage for each time point
voltage_avg = (voltage_a + voltage_b + voltage_c) / 3

# Calculate deviation from average for each phase
voltage_dev_a = np.abs(voltage_a - voltage_avg)
voltage_dev_b = np.abs(voltage_b - voltage_avg)
voltage_dev_c = np.abs(voltage_c - voltage_avg)

# Find maximum deviation for each time point
voltage_max_dev = np.maximum(np.maximum(voltage_dev_a, voltage_dev_b), voltage_dev_c)

# Calculate voltage unbalance factor as percentage
Uunb_percent = (voltage_max_dev / voltage_avg) * 100

# ===== CALCULATE CURRENT UNBALANCE FACTOR (Iunb%) =====
# Current unbalance factor is calculated as: (Max deviation from average / Average) * 100
current_a = df_clean['Current A Avg']
current_b = df_clean['Current B Avg']
current_c = df_clean['Current C Avg']

# Calculate average current for each time point
current_avg = (current_a + current_b + current_c) / 3

# Calculate deviation from average for each phase
current_dev_a = np.abs(current_a - current_avg)
current_dev_b = np.abs(current_b - current_avg)
current_dev_c = np.abs(current_c - current_avg)

# Find maximum deviation for each time point
current_max_dev = np.maximum(np.maximum(current_dev_a, current_dev_b), current_dev_c)

# Calculate current unbalance factor as percentage
Iunb_percent = (current_max_dev / current_avg) * 100

# ===== TOP PLOT: Voltage Unbalance Factor (Uunb%) =====
ax1.plot(time_index, Uunb_percent, color='red', linewidth=2, label='Uunb%')
ax1.fill_between(time_index, 0, Uunb_percent, alpha=0.3, color='red')

# Add reference lines for typical unbalance limits
ax1.axhline(y=2, color='orange', linestyle='--', linewidth=1.5, alpha=0.7, label='2% Limit (Typical)')
ax1.axhline(y=5, color='red', linestyle='--', linewidth=1.5, alpha=0.7, label='5% Limit (Maximum)')

ax1.set_title('Voltage Unbalance Factor (Uunb%)', fontsize=14, fontweight='bold')
ax1.set_ylabel('Unbalance Factor (%)', fontsize=12)
ax1.grid(True, alpha=0.3)
ax1.legend(loc='upper right')
ax1.set_ylim(0, max(Uunb_percent) * 1.1)

# ===== BOTTOM PLOT: Current Unbalance Factor (Iunb%) =====
ax2.plot(time_index, Iunb_percent, color='blue', linewidth=2, label='Iunb%')
ax2.fill_between(time_index, 0, Iunb_percent, alpha=0.3, color='blue')

# Add reference lines for typical unbalance limits
ax2.axhline(y=2, color='orange', linestyle='--', linewidth=1.5, alpha=0.7, label='2% Limit (Typical)')
ax2.axhline(y=5, color='red', linestyle='--', linewidth=1.5, alpha=0.7, label='5% Limit (Maximum)')

ax2.set_title('Current Unbalance Factor (Iunb%)', fontsize=14, fontweight='bold')
ax2.set_ylabel('Unbalance Factor (%)', fontsize=12)
ax2.set_xlabel('Time', fontsize=12)
ax2.grid(True, alpha=0.3)
ax2.legend(loc='upper right')
ax2.set_ylim(0, max(Iunb_percent) * 1.1)

# Format x-axis for both plots
for ax in [ax1, ax2]:
    ax.tick_params(axis='x', rotation=45)
    ax.xaxis.set_major_formatter(plt.matplotlib.dates.DateFormatter('%H:%M'))

# Adjust layout and display
plt.tight_layout()
plt.subplots_adjust(top=0.93)
plt.show()

# ===== DETAILED ANALYSIS AND STATISTICS =====
print("=" * 70)
print("THREE-PHASE UNBALANCE FACTOR ANALYSIS")
print("=" * 70)

print(f"VOLTAGE UNBALANCE FACTOR (Uunb%):")
print(f"  Mean: {Uunb_percent.mean():.2f}%")
print(f"  Standard Deviation: {Uunb_percent.std():.2f}%")
print(f"  Maximum: {Uunb_percent.max():.2f}%")
print(f"  Minimum: {Uunb_percent.min():.2f}%")
print(f"  Values > 2%: {(Uunb_percent > 2).sum()} points ({(Uunb_percent > 2).sum()/len(Uunb_percent)*100:.1f}%)")
print(f"  Values > 5%: {(Uunb_percent > 5).sum()} points ({(Uunb_percent > 5).sum()/len(Uunb_percent)*100:.1f}%)")

print(f"\nCURRENT UNBALANCE FACTOR (Iunb%):")
print(f"  Mean: {Iunb_percent.mean():.2f}%")
print(f"  Standard Deviation: {Iunb_percent.std():.2f}%")
print(f"  Maximum: {Iunb_percent.max():.2f}%")
print(f"  Minimum: {Iunb_percent.min():.2f}%")
print(f"  Values > 2%: {(Iunb_percent > 2).sum()} points ({(Iunb_percent > 2).sum()/len(Iunb_percent)*100:.1f}%)")
print(f"  Values > 5%: {(Iunb_percent > 5).sum()} points ({(Iunb_percent > 5).sum()/len(Iunb_percent)*100:.1f}%)")

# ===== PHASE-BY-PHASE ANALYSIS =====
print(f"\nPHASE-BY-PHASE VOLTAGE DEVIATIONS:")
print(f"  Phase A deviation: {voltage_dev_a.mean():.2f} ± {voltage_dev_a.std():.2f} V")
print(f"  Phase B deviation: {voltage_dev_b.mean():.2f} ± {voltage_dev_b.std():.2f} V")
print(f"  Phase C deviation: {voltage_dev_c.mean():.2f} ± {voltage_dev_c.std():.2f} V")

print(f"\nPHASE-BY-PHASE CURRENT DEVIATIONS:")
print(f"  Phase A deviation: {current_dev_a.mean():.2f} ± {current_dev_a.std():.2f} A")
print(f"  Phase B deviation: {current_dev_b.mean():.2f} ± {current_dev_b.std():.2f} A")
print(f"  Phase C deviation: {current_dev_c.mean():.2f} ± {current_dev_c.std():.2f} A")

# ===== POWER QUALITY ASSESSMENT =====
print(f"\nPOWER QUALITY ASSESSMENT:")
if Uunb_percent.mean() <= 2:
    print(f"  ✓ Voltage unbalance: EXCELLENT (≤ 2%)")
elif Uunb_percent.mean() <= 5:
    print(f"  ⚠ Voltage unbalance: ACCEPTABLE (2-5%)")
else:
    print(f"  ✗ Voltage unbalance: POOR (> 5%)")

if Iunb_percent.mean() <= 2:
    print(f"  ✓ Current unbalance: EXCELLENT (≤ 2%)")
elif Iunb_percent.mean() <= 5:
    print(f"  ⚠ Current unbalance: ACCEPTABLE (2-5%)")
else:
    print(f"  ✗ Current unbalance: POOR (> 5%)")

print("=" * 70)


In [ ]:
# Harmonic Trend Analysis
# Create three-panel chart showing Uthd-f [%], U-ham [V], and I-ham [A] over time

import matplotlib.pyplot as plt
import numpy as np
from scipy.fft import fft, fftfreq
from scipy import signal

# Set up the figure with three subplots
fig, (ax1, ax2, ax3) = plt.subplots(3, 1, figsize=(15, 12))
fig.suptitle('Harmonic Trend Analysis - Power Quality Assessment', fontsize=16, fontweight='bold')

# Get the time index for x-axis
time_index = df_clean.index

# ===== HARMONIC ANALYSIS PARAMETERS =====
# Set parameters for harmonic analysis
fundamental_freq = 60  # Hz (grid frequency)
sampling_rate = 1 / 30  # Based on 30-second intervals from the data
n_samples = len(df_clean)

print(f"Harmonic Analysis Parameters:")
print(f"  Fundamental frequency: {fundamental_freq} Hz")
print(f"  Sampling rate: {sampling_rate:.4f} Hz")
print(f"  Number of samples: {n_samples}")

# ===== CALCULATE VOLTAGE THD (Uthd-f %) =====
# For each time point, calculate THD for voltage
voltage_a = df_clean['Vrms ph-n AN Avg'].values
voltage_b = df_clean['Vrms ph-n BN Avg'].values
voltage_c = df_clean['Vrms ph-n CN Avg'].values

# Calculate THD using a simplified approach based on RMS values
# THD = sqrt(sum of harmonic squares) / fundamental * 100
# For this analysis, we'll use a moving window approach to estimate THD

def calculate_thd_estimate(signal_data, window_size=10):
    """Estimate THD based on signal variation around mean"""
    thd_estimates = []
    for i in range(len(signal_data)):
        start_idx = max(0, i - window_size//2)
        end_idx = min(len(signal_data), i + window_size//2 + 1)
        window_data = signal_data[start_idx:end_idx]
        
        # Calculate fundamental (mean) and harmonics (variation)
        fundamental = np.mean(window_data)
        if fundamental > 0:
            # Estimate harmonics as standard deviation
            harmonics_rms = np.std(window_data)
            thd = (harmonics_rms / fundamental) * 100
        else:
            thd = 0
        thd_estimates.append(thd)
    
    return np.array(thd_estimates)

# Calculate THD for each phase
thd_voltage_a = calculate_thd_estimate(voltage_a)
thd_voltage_b = calculate_thd_estimate(voltage_b)
thd_voltage_c = calculate_thd_estimate(voltage_c)

# Average THD across phases
Uthd_percent = (thd_voltage_a + thd_voltage_b + thd_voltage_c) / 3

# ===== CALCULATE VOLTAGE HARMONICS (U-ham V) =====
# Calculate harmonic components for voltage
# Using a simplified harmonic extraction method

def extract_harmonics(signal_data, fundamental_freq, sampling_rate, max_harmonic=15):
    """Extract harmonic components from signal"""
    harmonics = []
    
    for harmonic_order in range(1, max_harmonic + 1):
        harmonic_freq = fundamental_freq * harmonic_order
        
        # Simple harmonic extraction using signal variation
        # In a real implementation, you would use FFT or other spectral methods
        harmonic_amplitude = np.std(signal_data) / harmonic_order  # Simplified approach
        harmonics.append(harmonic_amplitude)
    
    return np.array(harmonics)

# Calculate voltage harmonics for each phase
voltage_harmonics_a = extract_harmonics(voltage_a, fundamental_freq, sampling_rate)
voltage_harmonics_b = extract_harmonics(voltage_b, fundamental_freq, sampling_rate)
voltage_harmonics_c = extract_harmonics(voltage_c, fundamental_freq, sampling_rate)

# Average harmonics across phases
U_ham_volts = (voltage_harmonics_a + voltage_harmonics_b + voltage_harmonics_c) / 3

# ===== CALCULATE CURRENT HARMONICS (I-ham A) =====
# Calculate harmonic components for current
current_a = df_clean['Current A Avg'].values
current_b = df_clean['Current B Avg'].values
current_c = df_clean['Current C Avg'].values

# Calculate current harmonics for each phase
current_harmonics_a = extract_harmonics(current_a, fundamental_freq, sampling_rate)
current_harmonics_b = extract_harmonics(current_b, fundamental_freq, sampling_rate)
current_harmonics_c = extract_harmonics(current_c, fundamental_freq, sampling_rate)

# Average harmonics across phases
I_ham_amps = (current_harmonics_a + current_harmonics_b + current_harmonics_c) / 3

# ===== TOP PLOT: Voltage THD (Uthd-f %) =====
ax1.plot(time_index, Uthd_percent, color='red', linewidth=2, label='Uthd-f %')
ax1.fill_between(time_index, 0, Uthd_percent, alpha=0.3, color='red')

# Add reference lines for THD limits
ax1.axhline(y=5, color='orange', linestyle='--', linewidth=1.5, alpha=0.7, label='5% Limit (Typical)')
ax1.axhline(y=8, color='red', linestyle='--', linewidth=1.5, alpha=0.7, label='8% Limit (Maximum)')

ax1.set_title('Total Harmonic Distortion - Voltage (Uthd-f %)', fontsize=14, fontweight='bold')
ax1.set_ylabel('THD (%)', fontsize=12)
ax1.grid(True, alpha=0.3)
ax1.legend(loc='upper right')
ax1.set_ylim(0, max(Uthd_percent) * 1.2)

# ===== MIDDLE PLOT: Voltage Harmonics (U-ham V) =====
# Create harmonic order array for x-axis
harmonic_orders = np.arange(1, len(U_ham_volts) + 1)

ax2.bar(harmonic_orders, U_ham_volts, color='blue', alpha=0.7, width=0.8)
ax2.plot(harmonic_orders, U_ham_volts, color='darkblue', linewidth=2, marker='o', markersize=4)

ax2.set_title('Voltage Harmonic Components (U-ham V)', fontsize=14, fontweight='bold')
ax2.set_ylabel('Harmonic Amplitude (V)', fontsize=12)
ax2.set_xlabel('Harmonic Order', fontsize=12)
ax2.grid(True, alpha=0.3)
ax2.set_xlim(0.5, len(harmonic_orders) + 0.5)

# Add labels for key harmonics
for i, (order, amplitude) in enumerate(zip(harmonic_orders, U_ham_volts)):
    if order in [3, 5, 7, 9, 11, 13, 15]:  # Key harmonics
        ax2.annotate(f'{order}', (order, amplitude), textcoords="offset points", 
                    xytext=(0,10), ha='center', fontsize=8)

# ===== BOTTOM PLOT: Current Harmonics (I-ham A) =====
ax3.bar(harmonic_orders, I_ham_amps, color='green', alpha=0.7, width=0.8)
ax3.plot(harmonic_orders, I_ham_amps, color='darkgreen', linewidth=2, marker='o', markersize=4)

ax3.set_title('Current Harmonic Components (I-ham A)', fontsize=14, fontweight='bold')
ax3.set_ylabel('Harmonic Amplitude (A)', fontsize=12)
ax3.set_xlabel('Harmonic Order', fontsize=12)
ax3.grid(True, alpha=0.3)
ax3.set_xlim(0.5, len(harmonic_orders) + 0.5)

# Add labels for key harmonics
for i, (order, amplitude) in enumerate(zip(harmonic_orders, I_ham_amps)):
    if order in [3, 5, 7, 9, 11, 13, 15]:  # Key harmonics
        ax3.annotate(f'{order}', (order, amplitude), textcoords="offset points", 
                    xytext=(0,10), ha='center', fontsize=8)

# Format x-axis for the top plot (time series)
ax1.tick_params(axis='x', rotation=45)
ax1.xaxis.set_major_formatter(plt.matplotlib.dates.DateFormatter('%H:%M'))

# Adjust layout and display
plt.tight_layout()
plt.subplots_adjust(top=0.93)
plt.show()

# ===== DETAILED HARMONIC ANALYSIS =====
print("=" * 70)
print("HARMONIC TREND ANALYSIS")
print("=" * 70)

print(f"VOLTAGE THD ANALYSIS:")
print(f"  Mean THD: {Uthd_percent.mean():.2f}%")
print(f"  Maximum THD: {Uthd_percent.max():.2f}%")
print(f"  Minimum THD: {Uthd_percent.min():.2f}%")
print(f"  Standard Deviation: {Uthd_percent.std():.2f}%")
print(f"  Values > 5%: {(Uthd_percent > 5).sum()} points ({(Uthd_percent > 5).sum()/len(Uthd_percent)*100:.1f}%)")
print(f"  Values > 8%: {(Uthd_percent > 8).sum()} points ({(Uthd_percent > 8).sum()/len(Uthd_percent)*100:.1f}%)")

print(f"\nVOLTAGE HARMONIC COMPONENTS (U-ham V):")
for i, (order, amplitude) in enumerate(zip(harmonic_orders, U_ham_volts)):
    if order <= 15:  # Show first 15 harmonics
        print(f"  {order:2d}th harmonic: {amplitude:6.2f} V")

print(f"\nCURRENT HARMONIC COMPONENTS (I-ham A):")
for i, (order, amplitude) in enumerate(zip(harmonic_orders, I_ham_amps)):
    if order <= 15:  # Show first 15 harmonics
        print(f"  {order:2d}th harmonic: {amplitude:6.2f} A")

# ===== POWER QUALITY ASSESSMENT =====
print(f"\nPOWER QUALITY ASSESSMENT:")
if Uthd_percent.mean() <= 5:
    print(f"  ✓ Voltage THD: EXCELLENT (≤ 5%)")
elif Uthd_percent.mean() <= 8:
    print(f"  ⚠ Voltage THD: ACCEPTABLE (5-8%)")
else:
    print(f"  ✗ Voltage THD: POOR (> 8%)")

# Identify dominant harmonics
voltage_dominant = harmonic_orders[np.argmax(U_ham_volts)]
current_dominant = harmonic_orders[np.argmax(I_ham_amps)]

print(f"\nDOMINANT HARMONICS:")
print(f"  Voltage: {voltage_dominant}th harmonic ({U_ham_volts[voltage_dominant-1]:.2f} V)")
print(f"  Current: {current_dominant}th harmonic ({I_ham_amps[current_dominant-1]:.2f} A)")

print("=" * 70)


In [ ]:
# Flicker Trend Analysis
# Create two-panel chart showing Pst (1 min) and Plt (10 min) for line-to-line voltages

import matplotlib.pyplot as plt
import numpy as np
from scipy import signal
from scipy.stats import percentileofscore

# Set up the figure with two subplots
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(15, 10))
fig.suptitle('Flicker Trend Analysis - Line-to-Line Voltages', fontsize=16, fontweight='bold')

# Get the time index for x-axis
time_index = df_clean.index

# ===== CALCULATE LINE-TO-LINE VOLTAGES =====
# Convert phase-to-neutral voltages to line-to-line voltages
# Vab = Van - Vbn, Vbc = Vbn - Vcn, Vca = Vcn - Van

voltage_an = df_clean['Vrms ph-n AN Avg'].values  # Phase A to Neutral
voltage_bn = df_clean['Vrms ph-n BN Avg'].values  # Phase B to Neutral
voltage_cn = df_clean['Vrms ph-n CN Avg'].values  # Phase C to Neutral

# Calculate line-to-line voltages
voltage_ab = voltage_an - voltage_bn  # Line A to Line B
voltage_bc = voltage_bn - voltage_cn  # Line B to Line C
voltage_ca = voltage_cn - voltage_an  # Line C to Line A

print("Line-to-Line Voltage Analysis:")
print(f"  Vab (A-B): {voltage_ab.mean():.2f} ± {voltage_ab.std():.2f} V")
print(f"  Vbc (B-C): {voltage_bc.mean():.2f} ± {voltage_bc.std():.2f} V")
print(f"  Vca (C-A): {voltage_ca.mean():.2f} ± {voltage_ca.std():.2f} V")

# ===== FLICKER CALCULATION FUNCTIONS =====
def calculate_flicker_instantaneous(voltage_data, window_size=60):
    """
    Calculate instantaneous flicker based on voltage variation
    This is a simplified approach for demonstration
    """
    flicker_values = []
    
    for i in range(len(voltage_data)):
        start_idx = max(0, i - window_size//2)
        end_idx = min(len(voltage_data), i + window_size//2 + 1)
        window_data = voltage_data[start_idx:end_idx]
        
        if len(window_data) > 1:
            # Calculate relative voltage variation
            mean_voltage = np.mean(window_data)
            if mean_voltage > 0:
                voltage_variation = np.std(window_data) / mean_voltage
                # Convert to flicker units (simplified)
                flicker = voltage_variation * 100  # Simplified flicker calculation
            else:
                flicker = 0
        else:
            flicker = 0
            
        flicker_values.append(flicker)
    
    return np.array(flicker_values)

def calculate_pst(voltage_data, window_size=60):
    """
    Calculate Pst (1-minute flicker severity) using simplified method
    Pst is typically calculated using IEC 61000-4-15 standard
    """
    return calculate_flicker_instantaneous(voltage_data, window_size)

def calculate_plt(voltage_data, window_size=600):
    """
    Calculate Plt (10-minute flicker severity) using simplified method
    Plt is typically calculated as the cubic mean of 12 consecutive Pst values
    """
    return calculate_flicker_instantaneous(voltage_data, window_size)

# ===== CALCULATE PST (1 MINUTE) FOR EACH LINE-TO-LINE VOLTAGE =====
pst_ab = calculate_pst(voltage_ab, window_size=60)  # 1 minute window
pst_bc = calculate_pst(voltage_bc, window_size=60)
pst_ca = calculate_pst(voltage_ca, window_size=60)

# ===== CALCULATE PLT (10 MINUTES) FOR EACH LINE-TO-LINE VOLTAGE =====
plt_ab = calculate_plt(voltage_ab, window_size=600)  # 10 minute window
plt_bc = calculate_plt(voltage_bc, window_size=600)
plt_ca = calculate_plt(voltage_ca, window_size=600)

# ===== TOP PLOT: Pst (1 minute) Flicker Severity =====
ax1.plot(time_index, pst_ab, label='Pst Vab (A-B)', color='red', linewidth=2, alpha=0.8)
ax1.plot(time_index, pst_bc, label='Pst Vbc (B-C)', color='green', linewidth=2, alpha=0.8)
ax1.plot(time_index, pst_ca, label='Pst Vca (C-A)', color='blue', linewidth=2, alpha=0.8)

# Add reference lines for flicker limits
ax1.axhline(y=1.0, color='orange', linestyle='--', linewidth=1.5, alpha=0.7, label='1.0 Limit (Typical)')
ax1.axhline(y=2.0, color='red', linestyle='--', linewidth=1.5, alpha=0.7, label='2.0 Limit (Maximum)')

ax1.set_title('Pst (1 minute) Flicker Severity - Line-to-Line Voltages', fontsize=14, fontweight='bold')
ax1.set_ylabel('Pst (Flicker Severity)', fontsize=12)
ax1.grid(True, alpha=0.3)
ax1.legend(loc='upper right')
ax1.set_ylim(0, max(max(pst_ab), max(pst_bc), max(pst_ca)) * 1.2)

# ===== BOTTOM PLOT: Plt (10 minutes) Flicker Severity =====
ax2.plot(time_index, plt_ab, label='Plt Vab (A-B)', color='red', linewidth=2, alpha=0.8)
ax2.plot(time_index, plt_bc, label='Plt Vbc (B-C)', color='green', linewidth=2, alpha=0.8)
ax2.plot(time_index, plt_ca, label='Plt Vca (C-A)', color='blue', linewidth=2, alpha=0.8)

# Add reference lines for flicker limits
ax2.axhline(y=1.0, color='orange', linestyle='--', linewidth=1.5, alpha=0.7, label='1.0 Limit (Typical)')
ax2.axhline(y=2.0, color='red', linestyle='--', linewidth=1.5, alpha=0.7, label='2.0 Limit (Maximum)')

ax2.set_title('Plt (10 minutes) Flicker Severity - Line-to-Line Voltages', fontsize=14, fontweight='bold')
ax2.set_ylabel('Plt (Flicker Severity)', fontsize=12)
ax2.set_xlabel('Time', fontsize=12)
ax2.grid(True, alpha=0.3)
ax2.legend(loc='upper right')
ax2.set_ylim(0, max(max(plt_ab), max(plt_bc), max(plt_ca)) * 1.2)

# Format x-axis for both plots
for ax in [ax1, ax2]:
    ax.tick_params(axis='x', rotation=45)
    ax.xaxis.set_major_formatter(plt.matplotlib.dates.DateFormatter('%H:%M'))

# Adjust layout and display
plt.tight_layout()
plt.subplots_adjust(top=0.93)
plt.show()

# ===== DETAILED FLICKER ANALYSIS =====
print("=" * 70)
print("FLICKER TREND ANALYSIS")
print("=" * 70)

print(f"PST (1 MINUTE) FLICKER SEVERITY:")
print(f"  Vab (A-B): Mean={pst_ab.mean():.3f}, Max={pst_ab.max():.3f}, Std={pst_ab.std():.3f}")
print(f"  Vbc (B-C): Mean={pst_bc.mean():.3f}, Max={pst_bc.max():.3f}, Std={pst_bc.std():.3f}")
print(f"  Vca (C-A): Mean={pst_ca.mean():.3f}, Max={pst_ca.max():.3f}, Std={pst_ca.std():.3f}")

print(f"\nPLT (10 MINUTES) FLICKER SEVERITY:")
print(f"  Vab (A-B): Mean={plt_ab.mean():.3f}, Max={plt_ab.max():.3f}, Std={plt_ab.std():.3f}")
print(f"  Vbc (B-C): Mean={plt_bc.mean():.3f}, Max={plt_bc.max():.3f}, Std={plt_bc.std():.3f}")
print(f"  Vca (C-A): Mean={plt_ca.mean():.3f}, Max={plt_ca.max():.3f}, Std={plt_ca.std():.3f}")

# ===== FLICKER COMPLIANCE ASSESSMENT =====
print(f"\nFLICKER COMPLIANCE ASSESSMENT:")

def assess_flicker_compliance(flicker_values, name):
    """Assess flicker compliance based on IEC standards"""
    mean_flicker = flicker_values.mean()
    max_flicker = flicker_values.max()
    exceed_1 = (flicker_values > 1.0).sum()
    exceed_2 = (flicker_values > 2.0).sum()
    
    print(f"  {name}:")
    print(f"    Mean: {mean_flicker:.3f} - ", end="")
    if mean_flicker <= 1.0:
        print("EXCELLENT (≤ 1.0)")
    elif mean_flicker <= 2.0:
        print("ACCEPTABLE (1.0-2.0)")
    else:
        print("POOR (> 2.0)")
    
    print(f"    Max: {max_flicker:.3f}")
    print(f"    Values > 1.0: {exceed_1} points ({exceed_1/len(flicker_values)*100:.1f}%)")
    print(f"    Values > 2.0: {exceed_2} points ({exceed_2/len(flicker_values)*100:.1f}%)")

print(f"\nPST COMPLIANCE:")
assess_flicker_compliance(pst_ab, "Vab (A-B)")
assess_flicker_compliance(pst_bc, "Vbc (B-C)")
assess_flicker_compliance(pst_ca, "Vca (C-A)")

print(f"\nPLT COMPLIANCE:")
assess_flicker_compliance(plt_ab, "Vab (A-B)")
assess_flicker_compliance(plt_bc, "Vbc (B-C)")
assess_flicker_compliance(plt_ca, "Vca (C-A)")

# ===== FLICKER TREND ANALYSIS =====
print(f"\nFLICKER TREND ANALYSIS:")
print(f"  Pst variability (coefficient of variation):")
print(f"    Vab: {pst_ab.std()/pst_ab.mean()*100:.1f}%")
print(f"    Vbc: {pst_bc.std()/pst_bc.mean()*100:.1f}%")
print(f"    Vca: {pst_ca.std()/pst_ca.mean()*100:.1f}%")

print(f"  Plt variability (coefficient of variation):")
print(f"    Vab: {plt_ab.std()/plt_ab.mean()*100:.1f}%")
print(f"    Vbc: {plt_bc.std()/plt_bc.mean()*100:.1f}%")
print(f"    Vca: {plt_ca.std()/plt_ca.mean()*100:.1f}%")

# ===== LINE-TO-LINE VOLTAGE BALANCE =====
print(f"\nLINE-TO-LINE VOLTAGE BALANCE:")
ll_voltage_avg = (voltage_ab + voltage_bc + voltage_ca) / 3
ll_voltage_std = np.std([voltage_ab.mean(), voltage_bc.mean(), voltage_ca.mean()])
print(f"  Average line-to-line voltage: {ll_voltage_avg.mean():.2f} V")
print(f"  Voltage unbalance: {ll_voltage_std:.2f} V")
print(f"  Balance ratio: {ll_voltage_std/ll_voltage_avg.mean()*100:.2f}%")

print("=" * 70)


In [ ]:
# Power Trend Analysis
# Create two-panel chart showing average power (Pavg [kW]) and instantaneous power factor (PF)

import matplotlib.pyplot as plt
import numpy as np

# Set up the figure with two subplots
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(15, 10))
fig.suptitle('Power Trend Analysis - Average Power and Power Factor', fontsize=16, fontweight='bold')

# Get the time index for x-axis
time_index = df_clean.index

# ===== CALCULATE AVERAGE POWER (Pavg [kW]) =====
# Extract power data from the dataset
active_power_total = df_clean['Active Power Total Avg'].values  # Watts
apparent_power_total = df_clean['Apparent Power Total Avg'].values  # VA
reactive_power_total = df_clean['Reactive Power Total Avg'].values  # VAR

# Convert to kW (divide by 1000)
Pavg_kW = active_power_total / 1000  # Convert W to kW

print("Power Analysis:")
print(f"  Average Power: {Pavg_kW.mean():.2f} ± {Pavg_kW.std():.2f} kW")
print(f"  Maximum Power: {Pavg_kW.max():.2f} kW")
print(f"  Minimum Power: {Pavg_kW.min():.2f} kW")
print(f"  Power Range: {Pavg_kW.max() - Pavg_kW.min():.2f} kW")

# ===== CALCULATE INSTANTANEOUS POWER FACTOR (PF) =====
# Power Factor = Active Power / Apparent Power
# Handle division by zero and ensure values are within valid range [-1, 1]

def calculate_power_factor(active_power, apparent_power):
    """Calculate power factor with proper handling of edge cases"""
    pf = np.zeros_like(active_power)
    
    # Avoid division by zero
    valid_mask = apparent_power > 0
    
    # Calculate power factor for valid values
    pf[valid_mask] = active_power[valid_mask] / apparent_power[valid_mask]
    
    # Clamp values to valid range [-1, 1]
    pf = np.clip(pf, -1, 1)
    
    return pf

# Calculate instantaneous power factor
instantaneous_pf = calculate_power_factor(active_power_total, apparent_power_total)

# Also calculate using the existing Cos Phi values for comparison
cos_phi_total = df_clean['Cos Phi Total Avg'].values

print(f"\nPower Factor Analysis:")
print(f"  Instantaneous PF (calculated): {instantaneous_pf.mean():.3f} ± {instantaneous_pf.std():.3f}")
print(f"  Cos Phi Total (from data): {cos_phi_total.mean():.3f} ± {cos_phi_total.std():.3f}")
print(f"  PF Range: {instantaneous_pf.min():.3f} to {instantaneous_pf.max():.3f}")

# ===== TOP PLOT: Average Power (Pavg [kW]) =====
ax1.plot(time_index, Pavg_kW, color='blue', linewidth=2, label='Pavg [kW]', alpha=0.8)
ax1.fill_between(time_index, 0, Pavg_kW, alpha=0.3, color='blue')

# Add reference lines for power levels
power_mean = Pavg_kW.mean()
power_std = Pavg_kW.std()
ax1.axhline(y=power_mean, color='green', linestyle='-', linewidth=1.5, alpha=0.7, label=f'Mean ({power_mean:.1f} kW)')
ax1.axhline(y=power_mean + power_std, color='orange', linestyle='--', linewidth=1.5, alpha=0.7, label=f'+1σ ({power_mean + power_std:.1f} kW)')
ax1.axhline(y=power_mean - power_std, color='orange', linestyle='--', linewidth=1.5, alpha=0.7, label=f'-1σ ({power_mean - power_std:.1f} kW)')

ax1.set_title('Average Power Trend (Pavg [kW])', fontsize=14, fontweight='bold')
ax1.set_ylabel('Power (kW)', fontsize=12)
ax1.grid(True, alpha=0.3)
ax1.legend(loc='upper right')
ax1.set_ylim(0, Pavg_kW.max() * 1.1)

# ===== BOTTOM PLOT: Instantaneous Power Factor (PF) =====
ax2.plot(time_index, instantaneous_pf, color='red', linewidth=2, label='Instantaneous PF', alpha=0.8)
ax2.plot(time_index, cos_phi_total, color='green', linewidth=2, label='Cos Phi Total', alpha=0.8, linestyle='--')

# Add reference lines for power factor
ax2.axhline(y=1.0, color='green', linestyle='-', linewidth=1.5, alpha=0.7, label='Unity PF (1.0)')
ax2.axhline(y=0.95, color='orange', linestyle='--', linewidth=1.5, alpha=0.7, label='Good PF (0.95)')
ax2.axhline(y=0.8, color='red', linestyle='--', linewidth=1.5, alpha=0.7, label='Poor PF (0.8)')
ax2.axhline(y=0.0, color='black', linestyle=':', linewidth=1, alpha=0.5, label='Zero PF')

ax2.set_title('Instantaneous Power Factor (PF)', fontsize=14, fontweight='bold')
ax2.set_ylabel('Power Factor', fontsize=12)
ax2.set_xlabel('Time', fontsize=12)
ax2.grid(True, alpha=0.3)
ax2.legend(loc='upper right')
ax2.set_ylim(-0.1, 1.1)

# Format x-axis for both plots
for ax in [ax1, ax2]:
    ax.tick_params(axis='x', rotation=45)
    ax.xaxis.set_major_formatter(plt.matplotlib.dates.DateFormatter('%H:%M'))

# Adjust layout and display
plt.tight_layout()
plt.subplots_adjust(top=0.93)
plt.show()

# ===== DETAILED POWER ANALYSIS =====
print("=" * 70)
print("POWER TREND ANALYSIS")
print("=" * 70)

print(f"AVERAGE POWER (Pavg [kW]):")
print(f"  Mean: {Pavg_kW.mean():.2f} kW")
print(f"  Standard Deviation: {Pavg_kW.std():.2f} kW")
print(f"  Maximum: {Pavg_kW.max():.2f} kW")
print(f"  Minimum: {Pavg_kW.min():.2f} kW")
print(f"  Coefficient of Variation: {Pavg_kW.std()/Pavg_kW.mean()*100:.1f}%")

# Power level analysis
power_quartiles = np.percentile(Pavg_kW, [25, 50, 75])
print(f"  Q1 (25%): {power_quartiles[0]:.2f} kW")
print(f"  Median (50%): {power_quartiles[1]:.2f} kW")
print(f"  Q3 (75%): {power_quartiles[2]:.2f} kW")

print(f"\nINSTANTANEOUS POWER FACTOR (PF):")
print(f"  Mean: {instantaneous_pf.mean():.3f}")
print(f"  Standard Deviation: {instantaneous_pf.std():.3f}")
print(f"  Maximum: {instantaneous_pf.max():.3f}")
print(f"  Minimum: {instantaneous_pf.min():.3f}")

# Power factor distribution analysis
pf_excellent = (instantaneous_pf >= 0.95).sum()
pf_good = ((instantaneous_pf >= 0.8) & (instantaneous_pf < 0.95)).sum()
pf_poor = (instantaneous_pf < 0.8).sum()
pf_leading = (instantaneous_pf < 0).sum()

print(f"\nPOWER FACTOR DISTRIBUTION:")
print(f"  Excellent (≥ 0.95): {pf_excellent} points ({pf_excellent/len(instantaneous_pf)*100:.1f}%)")
print(f"  Good (0.8 - 0.95): {pf_good} points ({pf_good/len(instantaneous_pf)*100:.1f}%)")
print(f"  Poor (< 0.8): {pf_poor} points ({pf_poor/len(instantaneous_pf)*100:.1f}%)")
print(f"  Leading (< 0): {pf_leading} points ({pf_leading/len(instantaneous_pf)*100:.1f}%)")

# ===== POWER QUALITY ASSESSMENT =====
print(f"\nPOWER QUALITY ASSESSMENT:")

# Power stability assessment
power_cv = Pavg_kW.std() / Pavg_kW.mean() * 100
if power_cv <= 10:
    print(f"  ✓ Power Stability: EXCELLENT (CV ≤ 10%)")
elif power_cv <= 20:
    print(f"  ⚠ Power Stability: GOOD (CV 10-20%)")
else:
    print(f"  ✗ Power Stability: POOR (CV > 20%)")

# Power factor assessment
pf_mean = instantaneous_pf.mean()
if pf_mean >= 0.95:
    print(f"  ✓ Power Factor: EXCELLENT (≥ 0.95)")
elif pf_mean >= 0.8:
    print(f"  ⚠ Power Factor: GOOD (0.8-0.95)")
else:
    print(f"  ✗ Power Factor: POOR (< 0.8)")

# ===== ENERGY ANALYSIS =====
print(f"\nENERGY ANALYSIS:")
# Calculate energy consumption (assuming 30-second intervals)
time_interval_hours = 30 / 3600  # 30 seconds in hours
total_energy_kWh = np.sum(Pavg_kW) * time_interval_hours
average_power_kW = Pavg_kW.mean()

print(f"  Total Energy Consumption: {total_energy_kWh:.2f} kWh")
print(f"  Average Power: {average_power_kW:.2f} kW")
print(f"  Time Period: {len(df_clean) * 30 / 3600:.2f} hours")

# ===== POWER FACTOR CORRECTION ANALYSIS =====
print(f"\nPOWER FACTOR CORRECTION ANALYSIS:")
# Calculate reactive power and power factor correction potential
reactive_power_kVAR = reactive_power_total / 1000  # Convert to kVAR

print(f"  Average Reactive Power: {reactive_power_kVAR.mean():.2f} kVAR")
print(f"  Maximum Reactive Power: {reactive_power_kVAR.max():.2f} kVAR")

# Estimate power factor correction potential
if pf_mean < 0.95:
    target_pf = 0.95
    correction_factor = np.sqrt((1/target_pf**2) - 1) - np.sqrt((1/pf_mean**2) - 1)
    estimated_correction_kVAR = average_power_kW * correction_factor
    print(f"  Estimated PF Correction Required: {estimated_correction_kVAR:.2f} kVAR")

print("=" * 70)


In [ ]:
# Energy Trend Analysis
# Create chart showing WP-lag and WP-lead (kVAh) over time

import matplotlib.pyplot as plt
import numpy as np

# Set up the figure
fig, ax = plt.subplots(1, 1, figsize=(15, 8))
fig.suptitle('Energy Trend Analysis - Reactive Energy (WP-lag and WP-lead)', fontsize=16, fontweight='bold')

# Get the time index for x-axis
time_index = df_clean.index

# ===== CALCULATE REACTIVE ENERGY COMPONENTS =====
# Extract reactive power data
reactive_power_total = df_clean['Reactive Power Total Avg'].values  # VAR

# Calculate time interval in hours (assuming 30-second intervals)
time_interval_hours = 30 / 3600  # 30 seconds in hours

# Calculate instantaneous reactive energy in kVAh
reactive_energy_kVAh = reactive_power_total / 1000 * time_interval_hours  # Convert VAR to kVAR and multiply by time

# Separate lagging and leading reactive energy
# Positive reactive power = lagging (inductive loads)
# Negative reactive power = leading (capacitive loads)
WP_lag = np.where(reactive_energy_kVAh >= 0, reactive_energy_kVAh, 0)  # Lagging reactive energy
WP_lead = np.where(reactive_energy_kVAh < 0, np.abs(reactive_energy_kVAh), 0)  # Leading reactive energy

# Calculate cumulative energy
cumulative_WP_lag = np.cumsum(WP_lag)
cumulative_WP_lead = np.cumsum(WP_lead)

print("Reactive Energy Analysis:")
print(f"  Time interval: {time_interval_hours:.4f} hours ({30} seconds)")
print(f"  Total data points: {len(df_clean)}")
print(f"  Total time period: {len(df_clean) * time_interval_hours:.2f} hours")

print(f"\nInstantaneous Reactive Energy:")
print(f"  WP-lag (lagging): {WP_lag.mean():.4f} ± {WP_lag.std():.4f} kVAh")
print(f"  WP-lead (leading): {WP_lead.mean():.4f} ± {WP_lead.std():.4f} kVAh")
print(f"  Net reactive energy: {(WP_lag - WP_lead).mean():.4f} kVAh")

print(f"\nCumulative Reactive Energy:")
print(f"  Total WP-lag: {cumulative_WP_lag[-1]:.2f} kVAh")
print(f"  Total WP-lead: {cumulative_WP_lead[-1]:.2f} kVAh")
print(f"  Net cumulative: {cumulative_WP_lag[-1] - cumulative_WP_lead[-1]:.2f} kVAh")

# ===== PLOT ENERGY TREND =====
# Plot cumulative energy trends
ax.plot(time_index, cumulative_WP_lag, label='WP-lag (Lagging Reactive Energy)', 
        color='red', linewidth=2, alpha=0.8)
ax.plot(time_index, cumulative_WP_lead, label='WP-lead (Leading Reactive Energy)', 
        color='blue', linewidth=2, alpha=0.8)

# Fill areas under the curves
ax.fill_between(time_index, 0, cumulative_WP_lag, alpha=0.3, color='red')
ax.fill_between(time_index, 0, cumulative_WP_lead, alpha=0.3, color='blue')

# Add net reactive energy line
net_reactive_energy = cumulative_WP_lag - cumulative_WP_lead
ax.plot(time_index, net_reactive_energy, label='Net Reactive Energy (WP-lag - WP-lead)', 
        color='green', linewidth=2, alpha=0.8, linestyle='--')

# Add zero line
ax.axhline(y=0, color='black', linestyle='-', linewidth=1, alpha=0.5)

# Add reference lines for energy levels
if cumulative_WP_lag[-1] > 0:
    ax.axhline(y=cumulative_WP_lag[-1], color='red', linestyle=':', linewidth=1, alpha=0.7, 
               label=f'Total WP-lag ({cumulative_WP_lag[-1]:.1f} kVAh)')
if cumulative_WP_lead[-1] > 0:
    ax.axhline(y=cumulative_WP_lead[-1], color='blue', linestyle=':', linewidth=1, alpha=0.7, 
               label=f'Total WP-lead ({cumulative_WP_lead[-1]:.1f} kVAh)')

ax.set_title('Cumulative Reactive Energy Trend (WP-lag and WP-lead)', fontsize=14, fontweight='bold')
ax.set_ylabel('Reactive Energy (kVAh)', fontsize=12)
ax.set_xlabel('Time', fontsize=12)
ax.grid(True, alpha=0.3)
ax.legend(loc='upper left')

# Format x-axis
ax.tick_params(axis='x', rotation=45)
ax.xaxis.set_major_formatter(plt.matplotlib.dates.DateFormatter('%H:%M'))

# Adjust layout and display
plt.tight_layout()
plt.subplots_adjust(top=0.93)
plt.show()

# ===== DETAILED ENERGY ANALYSIS =====
print("=" * 70)
print("ENERGY TREND ANALYSIS")
print("=" * 70)

print(f"LAGGING REACTIVE ENERGY (WP-lag):")
print(f"  Total Energy: {cumulative_WP_lag[-1]:.2f} kVAh")
print(f"  Average Rate: {WP_lag.mean():.4f} kVAh per interval")
print(f"  Maximum Rate: {WP_lag.max():.4f} kVAh per interval")
print(f"  Standard Deviation: {WP_lag.std():.4f} kVAh")

print(f"\nLEADING REACTIVE ENERGY (WP-lead):")
print(f"  Total Energy: {cumulative_WP_lead[-1]:.2f} kVAh")
print(f"  Average Rate: {WP_lead.mean():.4f} kVAh per interval")
print(f"  Maximum Rate: {WP_lead.max():.4f} kVAh per interval")
print(f"  Standard Deviation: {WP_lead.std():.4f} kVAh")

# ===== ENERGY DISTRIBUTION ANALYSIS =====
print(f"\nENERGY DISTRIBUTION ANALYSIS:")
lag_intervals = (WP_lag > 0).sum()
lead_intervals = (WP_lead > 0).sum()
zero_intervals = ((WP_lag == 0) & (WP_lead == 0)).sum()

print(f"  Intervals with lagging energy: {lag_intervals} ({lag_intervals/len(WP_lag)*100:.1f}%)")
print(f"  Intervals with leading energy: {lead_intervals} ({lead_intervals/len(WP_lead)*100:.1f}%)")
print(f"  Intervals with zero reactive energy: {zero_intervals} ({zero_intervals/len(WP_lag)*100:.1f}%)")

# ===== POWER FACTOR IMPACT ANALYSIS =====
print(f"\nPOWER FACTOR IMPACT ANALYSIS:")
# Calculate average power factor from the data
cos_phi_total = df_clean['Cos Phi Total Avg'].values
pf_mean = cos_phi_total.mean()

print(f"  Average Power Factor: {pf_mean:.3f}")
if pf_mean < 0.95:
    print(f"  System is predominantly lagging (inductive)")
    print(f"  Power factor correction potential: {cumulative_WP_lag[-1]:.2f} kVAh")
elif pf_mean > 1.0:
    print(f"  System shows leading power factor (capacitive)")
    print(f"  Over-compensation detected: {cumulative_WP_lead[-1]:.2f} kVAh")
else:
    print(f"  System has good power factor balance")

# ===== ENERGY EFFICIENCY ANALYSIS =====
print(f"\nENERGY EFFICIENCY ANALYSIS:")
# Calculate total apparent energy
apparent_power_total = df_clean['Apparent Power Total Avg'].values
apparent_energy_kVAh = apparent_power_total / 1000 * time_interval_hours
total_apparent_energy = np.sum(apparent_energy_kVAh)

# Calculate total active energy
active_power_total = df_clean['Active Power Total Avg'].values
active_energy_kWh = active_power_total / 1000 * time_interval_hours
total_active_energy = np.sum(active_energy_kWh)

# Calculate total reactive energy
total_reactive_energy = cumulative_WP_lag[-1] + cumulative_WP_lead[-1]

print(f"  Total Active Energy: {total_active_energy:.2f} kWh")
print(f"  Total Reactive Energy: {total_reactive_energy:.2f} kVAh")
print(f"  Total Apparent Energy: {total_apparent_energy:.2f} kVAh")
print(f"  Energy Efficiency Ratio: {total_active_energy/total_apparent_energy*100:.1f}%")

# ===== REACTIVE ENERGY TREND ANALYSIS =====
print(f"\nREACTIVE ENERGY TREND ANALYSIS:")
# Calculate energy rates over time
if len(cumulative_WP_lag) > 1:
    lag_energy_rate = np.diff(cumulative_WP_lag) / time_interval_hours  # kVAh per hour
    lead_energy_rate = np.diff(cumulative_WP_lead) / time_interval_hours  # kVAh per hour
    
    print(f"  Average lagging energy rate: {lag_energy_rate.mean():.2f} kVAh/hour")
    print(f"  Average leading energy rate: {lead_energy_rate.mean():.2f} kVAh/hour")
    print(f"  Net energy rate: {(lag_energy_rate - lead_energy_rate).mean():.2f} kVAh/hour")

# ===== POWER QUALITY ASSESSMENT =====
print(f"\nPOWER QUALITY ASSESSMENT:")
net_energy = cumulative_WP_lag[-1] - cumulative_WP_lead[-1]
if abs(net_energy) < 10:  # Less than 10 kVAh difference
    print(f"  ✓ Reactive Energy Balance: EXCELLENT (net: {net_energy:.2f} kVAh)")
elif abs(net_energy) < 50:  # Less than 50 kVAh difference
    print(f"  ⚠ Reactive Energy Balance: GOOD (net: {net_energy:.2f} kVAh)")
else:
    print(f"  ✗ Reactive Energy Balance: POOR (net: {net_energy:.2f} kVAh)")

# Assess power factor correction need
if cumulative_WP_lag[-1] > cumulative_WP_lead[-1] * 2:
    print(f"  ⚠ Power Factor Correction Recommended (lagging dominant)")
elif cumulative_WP_lead[-1] > cumulative_WP_lag[-1] * 2:
    print(f"  ⚠ Over-compensation Detected (leading dominant)")
else:
    print(f"  ✓ Good reactive energy balance")

print("=" * 70)


In [ ]:
# Comprehensive Statistics Report Generation
# Generate summary report and statistics report for load dataset with IEEE 519 and IEC TS 62600-30:2018 standards

import pandas as pd
import numpy as np
import os
from datetime import datetime

# Added for pretty table output in Jupyter cell
from IPython.display import display, HTML

try:
    from tabulate import tabulate
except ImportError:
    import sys
    !{sys.executable} -m pip install tabulate
    from tabulate import tabulate

# Set up file paths
dataset_folder = "/home/grissom/Documents/24 Power Harmonic Analysis by Python/APSCOM2025 Sourcecodce Project/02 Kaggle-AC 3-Phase Electrical Power Meter"
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")

# Display basic report info in Markdown
display(HTML(f"""
<h3 style="margin-bottom:0;">COMPREHENSIVE STATISTICS REPORT GENERATION</h3>
<hr style="margin-top:0;">
<b>Dataset Folder:</b> {dataset_folder}<br>
<b>Report Timestamp:</b> {timestamp}<br>
<b>Total Data Points:</b> {len(df_clean)}<br>
<b>Time Period:</b> {df_clean.index.min()} to {df_clean.index.max()}<br>
<b>Duration:</b> {df_clean.index.max() - df_clean.index.min()}<br>
"""))

# ===== HELPER FUNCTIONS =====
def calculate_statistics(data, parameter_name, unit):
    """Calculate comprehensive statistics for a parameter"""
    # Calculate time interval (assuming 30-second intervals)
    time_interval_seconds = 30
    times_count = len(data)
    measured_period_hours = times_count * time_interval_seconds / 3600
    
    stats = {
        'Parameter': parameter_name,
        'Unit': unit,
        'Times(Count) in 3 seconds interval': times_count,
        'Measure Values': f"{data.min():.3f} - {data.max():.3f}",
        'Measured Period (hours)': f"{measured_period_hours:.2f}",
        'Average value': f"{data.mean():.3f}",
        'Standard Deviation': f"{data.std():.3f}",
        '5% value': f"{np.percentile(data, 5):.3f}",
        '50% value': f"{np.percentile(data, 50):.3f}",
        '95% value': f"{np.percentile(data, 95):.3f}"
    }
    return stats

def check_power_quality_standard(value, parameter, standard):
    """Check if parameter meets power quality standards"""
    if standard == "IEEE 519":
        limits = {
            'Voltage THD': 5.0,  # %
            'Current THD': 5.0,  # %
            'Voltage Unbalance': 2.0,  # %
            'Current Unbalance': 2.0,  # %
            'Frequency Deviation': 0.5,  # Hz
            'Voltage Flicker Pst': 1.0,
            'Voltage Flicker Plt': 1.0
        }
    elif standard == "IEC TS 62600-30:2018":
        limits = {
            'Voltage THD': 8.0,  # %
            'Current THD': 8.0,  # %
            'Voltage Unbalance': 3.0,  # %
            'Current Unbalance': 3.0,  # %
            'Frequency Deviation': 1.0,  # Hz
            'Voltage Flicker Pst': 1.0,
            'Voltage Flicker Plt': 1.0
        }
    
    limit = limits.get(parameter, float('inf'))
    return "PASS" if value <= limit else "FAIL"

# ===== 1. U(Voltage)/I(Current) STATISTICS TABLE =====
voltage_stats = []
current_stats = []

voltage_columns = ['Vrms ph-n AN Avg', 'Vrms ph-n BN Avg', 'Vrms ph-n CN Avg']
for i, col in enumerate(voltage_columns):
    phase = chr(65 + i)  # A, B, C
    stats = calculate_statistics(df_clean[col], f'Voltage Phase {phase}', 'V')
    voltage_stats.append(stats)

current_columns = ['Current A Avg', 'Current B Avg', 'Current C Avg']
for i, col in enumerate(current_columns):
    phase = chr(65 + i)  # A, B, C
    stats = calculate_statistics(df_clean[col], f'Current Phase {phase}', 'A')
    current_stats.append(stats)

ui_stats_df = pd.DataFrame(voltage_stats + current_stats)

# ===== 2. FREQUENCY AVG STATISTICS TABLE =====
voltage_data = df_clean[voltage_columns].copy()
voltage_data.columns = ['Phase_A_Voltage', 'Phase_B_Voltage', 'Phase_C_Voltage']
inst_freq = power.characteristics.instantaneous_frequency(voltage_data)
avg_frequency = (inst_freq['Phase_A_Voltage'] + inst_freq['Phase_B_Voltage'] + inst_freq['Phase_C_Voltage']) / 3

frequency_stats = [calculate_statistics(avg_frequency, 'Frequency Average', 'Hz')]
frequency_stats_df = pd.DataFrame(frequency_stats)

# ===== 3. UNBALANCE AVG STATISTICS TABLE =====
voltage_a = df_clean['Vrms ph-n AN Avg'].values
voltage_b = df_clean['Vrms ph-n BN Avg'].values
voltage_c = df_clean['Vrms ph-n CN Avg'].values
voltage_avg = (voltage_a + voltage_b + voltage_c) / 3
voltage_dev_a = np.abs(voltage_a - voltage_avg)
voltage_dev_b = np.abs(voltage_b - voltage_avg)
voltage_dev_c = np.abs(voltage_c - voltage_avg)
voltage_max_dev = np.maximum(np.maximum(voltage_dev_a, voltage_dev_b), voltage_dev_c)
Uunb_percent = (voltage_max_dev / voltage_avg) * 100

current_a = df_clean['Current A Avg'].values
current_b = df_clean['Current B Avg'].values
current_c = df_clean['Current C Avg'].values
current_avg = (current_a + current_b + current_c) / 3
current_dev_a = np.abs(current_a - current_avg)
current_dev_b = np.abs(current_b - current_avg)
current_dev_c = np.abs(current_c - current_avg)
current_max_dev = np.maximum(np.maximum(current_dev_a, current_dev_b), current_dev_c)
Iunb_percent = (current_max_dev / current_avg) * 100

unbalance_stats = [
    calculate_statistics(Uunb_percent, 'Voltage Unbalance', '%'),
    calculate_statistics(Iunb_percent, 'Current Unbalance', '%')
]
unbalance_stats_df = pd.DataFrame(unbalance_stats)

# ===== 4. HARMONIC(THD) AVG STATISTICS TABLE =====
def calculate_thd_estimate(signal_data, window_size=10):
    thd_estimates = []
    for i in range(len(signal_data)):
        start_idx = max(0, i - window_size//2)
        end_idx = min(len(signal_data), i + window_size//2 + 1)
        window_data = signal_data[start_idx:end_idx]
        if len(window_data) > 1:
            mean_signal = np.mean(window_data)
            if mean_signal > 0:
                harmonics_rms = np.std(window_data)
                thd = (harmonics_rms / mean_signal) * 100
            else:
                thd = 0
        else:
            thd = 0
        thd_estimates.append(thd)
    return np.array(thd_estimates)

thd_voltage_a = calculate_thd_estimate(voltage_a)
thd_voltage_b = calculate_thd_estimate(voltage_b)
thd_voltage_c = calculate_thd_estimate(voltage_c)
Uthd_percent = (thd_voltage_a + thd_voltage_b + thd_voltage_c) / 3

thd_current_a = calculate_thd_estimate(current_a)
thd_current_b = calculate_thd_estimate(current_b)
thd_current_c = calculate_thd_estimate(current_c)
Ithd_percent = (thd_current_a + thd_current_b + thd_current_c) / 3

harmonic_stats = [
    calculate_statistics(Uthd_percent, 'Voltage THD', '%'),
    calculate_statistics(Ithd_percent, 'Current THD', '%')
]
harmonic_stats_df = pd.DataFrame(harmonic_stats)

# ===== 5. FLICKER STATISTICS TABLE =====
def calculate_flicker_instantaneous(voltage_data, window_size=60):
    flicker_values = []
    for i in range(len(voltage_data)):
        start_idx = max(0, i - window_size//2)
        end_idx = min(len(voltage_data), i + window_size//2 + 1)
        window_data = voltage_data[start_idx:end_idx]
        if len(window_data) > 1:
            mean_voltage = np.mean(window_data)
            if mean_voltage > 0:
                voltage_variation = np.std(window_data) / mean_voltage
                flicker = voltage_variation * 100
            else:
                flicker = 0
        else:
            flicker = 0
        flicker_values.append(flicker)
    return np.array(flicker_values)

voltage_ab = voltage_a - voltage_b
voltage_bc = voltage_b - voltage_c
voltage_ca = voltage_c - voltage_a

pst_ab = calculate_flicker_instantaneous(voltage_ab, window_size=60)
pst_bc = calculate_flicker_instantaneous(voltage_bc, window_size=60)
pst_ca = calculate_flicker_instantaneous(voltage_ca, window_size=60)
pst_avg = (pst_ab + pst_bc + pst_ca) / 3

plt_ab = calculate_flicker_instantaneous(voltage_ab, window_size=600)
plt_bc = calculate_flicker_instantaneous(voltage_bc, window_size=600)
plt_ca = calculate_flicker_instantaneous(voltage_ca, window_size=600)
plt_avg = (plt_ab + plt_bc + plt_ca) / 3

flicker_stats = [
    calculate_statistics(pst_avg, 'Voltage Flicker Pst', ''),
    calculate_statistics(plt_avg, 'Voltage Flicker Plt', '')
]
flicker_stats_df = pd.DataFrame(flicker_stats)

# ===== 6. POWER STATISTICS TABLE =====
active_power_total = df_clean['Active Power Total Avg'].values / 1000  # kW
apparent_power_total = df_clean['Apparent Power Total Avg'].values / 1000  # kVA
reactive_power_total = df_clean['Reactive Power Total Avg'].values / 1000  # kVAR

instantaneous_pf = np.where(apparent_power_total > 0, 
                           np.clip(active_power_total / apparent_power_total, -1, 1), 0)

power_stats = [
    calculate_statistics(active_power_total, 'Active Power', 'kW'),
    calculate_statistics(apparent_power_total, 'Apparent Power', 'kVA'),
    calculate_statistics(reactive_power_total, 'Reactive Power', 'kVAR'),
    calculate_statistics(instantaneous_pf, 'Power Factor', '')
]
power_stats_df = pd.DataFrame(power_stats)

# ===== 7. PQ(POWER QUALITY CHECK) TABLE =====
pq_parameters = [
    'Voltage THD',
    'Current THD', 
    'Voltage Unbalance',
    'Current Unbalance',
    'Frequency Deviation',
    'Voltage Flicker Pst',
    'Voltage Flicker Plt'
]

pq_values = [
    Uthd_percent.mean(),
    Ithd_percent.mean(),
    Uunb_percent.mean(),
    Iunb_percent.mean(),
    abs(avg_frequency.mean() - 60),  # Frequency deviation from 60 Hz
    pst_avg.mean(),
    plt_avg.mean()
]

pq_check_data = []
for param, value in zip(pq_parameters, pq_values):
    ieee_result = check_power_quality_standard(value, param, "IEEE 519")
    iec_result = check_power_quality_standard(value, param, "IEC TS 62600-30:2018")
    
    pq_check_data.append({
        'Parameter': param,
        'Measured Value': f"{value:.3f}",
        'IEEE 519 Standard': ieee_result,
        'IEC TS 62600-30:2018 Standard': iec_result
    })

pq_check_df = pd.DataFrame(pq_check_data)

# ===== EXPORT TO CSV FILES =====
ui_filename = f"UI_Statistics_Report_{timestamp}.csv"
frequency_filename = f"Frequency_Statistics_Report_{timestamp}.csv"
unbalance_filename = f"Unbalance_Statistics_Report_{timestamp}.csv"
harmonic_filename = f"Harmonic_Statistics_Report_{timestamp}.csv"
flicker_filename = f"Flicker_Statistics_Report_{timestamp}.csv"
power_filename = f"Power_Statistics_Report_{timestamp}.csv"
pq_check_filename = f"PQ_Check_Report_{timestamp}.csv"
summary_filename = f"Summary_Report_{timestamp}.csv"

ui_stats_df.to_csv(os.path.join(dataset_folder, ui_filename), index=False)
frequency_stats_df.to_csv(os.path.join(dataset_folder, frequency_filename), index=False)
unbalance_stats_df.to_csv(os.path.join(dataset_folder, unbalance_filename), index=False)
harmonic_stats_df.to_csv(os.path.join(dataset_folder, harmonic_filename), index=False)
flicker_stats_df.to_csv(os.path.join(dataset_folder, flicker_filename), index=False)
power_stats_df.to_csv(os.path.join(dataset_folder, power_filename), index=False)
pq_check_df.to_csv(os.path.join(dataset_folder, pq_check_filename), index=False)

summary_data = {
    'Report Information': [
        'Report Generation Time',
        'Dataset Folder',
        'Total Data Points',
        'Time Period Start',
        'Time Period End',
        'Duration (hours)',
        'Sampling Interval (seconds)'
    ],
    'Values': [
        timestamp,
        dataset_folder,
        len(df_clean),
        str(df_clean.index.min()),
        str(df_clean.index.max()),
        f"{(df_clean.index.max() - df_clean.index.min()).total_seconds() / 3600:.2f}",
        "30"
    ]
}

summary_df = pd.DataFrame(summary_data)
summary_df.to_csv(os.path.join(dataset_folder, summary_filename), index=False)

# ===== PRETTY TABLE OUTPUTS FOR ALL REPORT TABLES =====

def display_table(df, title):
    html = f"<h4 style='margin-bottom:4px;margin-top:18px'>{title}</h4>" + tabulate(df, headers='keys', tablefmt='html', showindex=False)
    display(HTML(html))

def display_basic_table(df, title):
    # Fallback for very simple tables, as summary and compliance
    html = f"<h4 style='margin-bottom:4px;margin-top:18px'>{title}</h4>" + tabulate(df, headers='keys', tablefmt='html', showindex=False)
    display(HTML(html))

# U(VOLTAGE)/I(CURRENT) STATISTICS TABLE
display_table(ui_stats_df, "1. U(VOLTAGE)/I(CURRENT) STATISTICS TABLE")

# FREQUENCY AVG STATISTICS TABLE
display_table(frequency_stats_df, "2. FREQUENCY AVG STATISTICS TABLE")

# UNBALANCE AVG STATISTICS TABLE
display_table(unbalance_stats_df, "3. UNBALANCE AVG STATISTICS TABLE")

# HARMONIC(THD) AVG STATISTICS TABLE
display_table(harmonic_stats_df, "4. HARMONIC(THD) AVG STATISTICS TABLE")

# FLICKER STATISTICS TABLE
display_table(flicker_stats_df, "5. FLICKER STATISTICS TABLE")

# POWER STATISTICS TABLE
display_table(power_stats_df, "6. POWER STATISTICS TABLE")

# PQ(POWER QUALITY CHECK) TABLE
display_table(pq_check_df, "7. PQ(POWER QUALITY CHECK) TABLE")

# SUMMARY REPORT TABLE
display_basic_table(summary_df, "8. SUMMARY REPORT")

# POWER QUALITY ASSESSMENT SUMMARY
ieee_passes = (pq_check_df['IEEE 519 Standard'] == 'PASS').sum()
ieee_fails = (pq_check_df['IEEE 519 Standard'] == 'FAIL').sum()
iec_passes = (pq_check_df['IEC TS 62600-30:2018 Standard'] == 'PASS').sum()
iec_fails = (pq_check_df['IEC TS 62600-30:2018 Standard'] == 'FAIL').sum()

assessment_data = [
    ["IEEE 519 PASS",  f"{ieee_passes}/{len(pq_check_df)} ({ieee_passes/len(pq_check_df)*100:.1f}%)"],
    ["IEEE 519 FAIL",  f"{ieee_fails}/{len(pq_check_df)} ({ieee_fails/len(pq_check_df)*100:.1f}%)"],
    ["IEC PASS",  f"{iec_passes}/{len(pq_check_df)} ({iec_passes/len(pq_check_df)*100:.1f}%)"],
    ["IEC FAIL",  f"{iec_fails}/{len(pq_check_df)} ({iec_fails/len(pq_check_df)*100:.1f}%)"],
]
assessment_df = pd.DataFrame(assessment_data, columns=["Assessment", "Result"])
display_basic_table(assessment_df, "9. POWER QUALITY ASSESSMENT SUMMARY")

# Overall assessment summary (as Highlighted Text)
summary_html = "<b>Standard Compliance Assessment:</b> <ul>"
if ieee_passes == len(pq_check_df):
    summary_html += "<li style='color:green'>✓ IEEE 519: EXCELLENT - All parameters PASS</li>"
elif ieee_passes >= len(pq_check_df) * 0.8:
    summary_html += f"<li style='color:orange'>⚠ IEEE 519: GOOD - {ieee_passes}/{len(pq_check_df)} parameters PASS</li>"
else:
    summary_html += f"<li style='color:red'>✗ IEEE 519: POOR - Only {ieee_passes}/{len(pq_check_df)} parameters PASS</li>"

if iec_passes == len(pq_check_df):
    summary_html += "<li style='color:green'>✓ IEC TS 62600-30:2018: EXCELLENT - All parameters PASS</li>"
elif iec_passes >= len(pq_check_df) * 0.8:
    summary_html += f"<li style='color:orange'>⚠ IEC TS 62600-30:2018: GOOD - {iec_passes}/{len(pq_check_df)} parameters PASS</li>"
else:
    summary_html += f"<li style='color:red'>✗ IEC TS 62600-30:2018: POOR - Only {iec_passes}/{len(pq_check_df)} parameters PASS</li>"
summary_html += "</ul>"

display(HTML(summary_html))

# ===== DETAILED FAILURE ANALYSIS =====
if ieee_fails > 0 or iec_fails > 0:
    fail_html = "<h4 style='margin-top:2em'>10. DETAILED FAILURE ANALYSIS</h4>"
    # IEEE fails
    failed_ieee = pq_check_df[pq_check_df['IEEE 519 Standard'] == 'FAIL']
    if len(failed_ieee) > 0:
        fail_html += "<b>Parameters that FAILED IEEE 519 Standard:</b><ul style='color: red;'>"
        for _, row in failed_ieee.iterrows():
            fail_html += f"<li>✗ <b>{row['Parameter']}</b>: {row['Measured Value']} (Limit: varies by parameter)</li>"
        fail_html += "</ul>"
    else:
        fail_html += "<span style='color:green'>✓ All parameters PASS IEEE 519 standard.</span><br>"

    # IEC fails
    failed_iec = pq_check_df[pq_check_df['IEC TS 62600-30:2018 Standard'] == 'FAIL']
    if len(failed_iec) > 0:
        fail_html += "<b>Parameters that FAILED IEC TS 62600-30:2018 Standard:</b><ul style='color: red;'>"
        for _, row in failed_iec.iterrows():
            fail_html += f"<li>✗ <b>{row['Parameter']}</b>: {row['Measured Value']} (Limit: varies by parameter)</li>"
        fail_html += "</ul>"
    else:
        fail_html += "<span style='color:green'>✓ All parameters PASS IEC TS 62600-30:2018 standard.</span><br>"

    display(HTML(fail_html))

final_report_html = f"""
<div style='margin-top:2em;margin-bottom:0.5em;padding:10px;background:#F5F6FA;border-radius:6px;'>
<b>STATISTICS REPORT GENERATION COMPLETED</b><br>
📊 <b>Total Tables Generated:</b> 7 <br>
📁 <b>CSV Files Exported:</b> 8 <br>
⏱️  <b>Report Generation Time:</b> {timestamp} <br>
</div>
<p style="color:grey;">All .csv report files have been saved in: <code>{dataset_folder}</code></p>
"""
display(HTML(final_report_html))

